# Addressing Data Spilling

The sales.xlsx contains historical sales data about different customer purchases in stores.
Your current team is only interested in the following product types:
Climbing Accessories, Cooking Gear, First Aid, Golf Accessories, Insect Repellents, and Sleeping Bags.
You need to read the files into pandas DataFrames and prepare the output so that it can be added into your analytics pipeline.

In [2]:
import pandas as pd
import copy

In [3]:
sales_data = pd.read_excel("sales.xlsx")
sales_data.head()

,Year,Product,line,Product.1,type,Product.2,Order,method,type.1,Retailer,...,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1
0,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,...,6.59,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,...,7145.88,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Camping,Equipment,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,...,91707.18,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sales_data.shape

(65535, 26)

In [5]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65535 entries, 0 to 65534
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       65535 non-null  int64  
 1   Product    65535 non-null  object 
 2   line       65535 non-null  object 
 3   Product.1  65535 non-null  object 
 4   type       65535 non-null  object 
 5   Product.2  65535 non-null  object 
 6   Order      65535 non-null  object 
 7   method     61083 non-null  object 
 8   type.1     46709 non-null  object 
 9   Retailer   32525 non-null  object 
 10  country    24135 non-null  object 
 11  Revenue    21494 non-null  object 
 12  Planned    20925 non-null  object 
 13  revenue    20645 non-null  object 
 14  Product.3  20588 non-null  object 
 15  cost       19224 non-null  float64
 16  Quantity   13663 non-null  float64
 17  Unit       7046 non-null   float64
 18  cost.1     2403 non-null   float64
 19  Unit.1     650 non-null    float64
 20  price 

To look at the data types of the columns

In [6]:
sales_data.dtypes

Year           int64
Product       object
line          object
Product.1     object
type          object
Product.2     object
Order         object
method        object
type.1        object
Retailer      object
country       object
Revenue       object
Planned       object
revenue       object
Product.3     object
cost         float64
Quantity     float64
Unit         float64
cost.1       float64
Unit.1       float64
price        float64
Gross        float64
profit       float64
Unit.2       float64
sale         float64
price.1      float64
dtype: object

In [7]:
for label, content in sales_data.iteritems():
    print(label, content[1])

Year 2004
Product Camping
line Equipment
Product.1 Cooking
type Gear
Product.2 TrailChef
Order Water
method Bag
type.1 Telephone
Retailer Canada
country 13444.68
Revenue 14313.48
Planned 6298.8
revenue 2172
Product.3 2.9
cost 6.59
Quantity 7145.88
Unit 6.19
cost.1 nan
Unit.1 nan
price nan
Gross nan
profit nan
Unit.2 nan
sale nan
price.1 nan


In [8]:
# There seems to be some column spillage in our data. The data should appear as shown below:
d = pd.DataFrame.from_dict(
    {
    "Year": [2004],
    "Product line": ["Camping Equipment"],
    "Product type": ["Cooking Gear"],
    "Product": ["TrailChef Water Bag"],
    "Order method type": ["Telephone"],
    "Retailer Country": ["Canada"],
    "Revenue": [13444.68],
    "Planned revenue": [14313.48],
    "Product cost": [6298.8],
    "Quantity": [2172],
    "Unit cost": [2.9],
    "Unit price": [6.59],
    "Gross Profit": [7145.88],
    "Unit sale price": [6.19]
})

This is how data should look like:

In [9]:
d

,Year,Product line,Product type,Product,Order method type,Retailer Country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross Profit,Unit sale price
0,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Canada,13444.68,14313.48,6298.8,2172,2.9,6.59,7145.88,6.19


Let's fix the spillage on the product line column which currently is as product and line separate columns

In [10]:
sales_data.groupby(["Product", "line"])["Year"].count()

Product         line       
Camping         Equipment      20112
Golf            Equipment       6615
Mountaineering  Equipment       9261
Outdoor         Protection      6615
Personal        Accessories    22932
Name: Year, dtype: int64

In [11]:
sales_data["Product line"] = sales_data.apply(lambda x: x["Product"] + " " + x["line"], axis=1)

In [12]:
sales_data = sales_data.drop(["Product", "line"], axis=1)
sales_data.head()

,Year,Product.1,type,Product.2,Order,method,type.1,Retailer,country,Revenue,...,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line
0,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,States,315044,...,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
1,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,...,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
2,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
3,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
4,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,181120,235237,...,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment


Now let's make a copy of our data

In [20]:
tmp = copy.deepcopy(sales_data)
tmp.head()

,Year,Product.1,type,Product.2,Order,method,type.1,Retailer,country,Revenue,...,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line
0,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,States,315044,...,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
1,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,...,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
2,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
3,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
4,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,181120,235237,...,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment


We proceed at looking at the next column, Product.1 and type

In [21]:
tmp.groupby(["Product.1", "type"])["Year"].count()

Product.1   type       
Binoculars  Opera           441
            Ranger          441
            Seeker         1764
Climbing    Accessories    3087
Cooking     Gear           5880
Eyewear     Bella           441
            Capri           441
            Cat             441
            Dante           441
            Fairway         441
            Hawk            441
            Inferno         441
            Maximus         441
            Polar          2205
            Retro           441
            Trendi          441
            Zone            441
First       Aid            2205
Golf        Accessories    1764
Insect      Repellents     2205
Irons       Hailstorm       882
            Lady            882
Knives      Bear            882
            Double          441
            Edge            441
            Max             441
            Pocket          441
            Single          441
Lanterns    EverGlow       2205
            Firefly        2646
            Flic

In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tmp.groupby(["Product.1", "type", "Product.2"])["Year"].count())

Product.1   type         Product.2  
Binoculars  Opera        Vision          441
            Ranger       Vision          441
            Seeker       35              441
                         50              441
                         Extreme         441
                         Mini            441
Climbing    Accessories  Firefly        1323
                         Granite        1764
Cooking     Gear         TrailChef      5880
Eyewear     Bella        E-mail           63
                         Fax              63
                         Mail             63
                         Sales            63
                         Special          63
                         Telephone        63
                         Web              63
            Capri        E-mail           63
                         Fax              63
                         Mail             63
                         Sales            63
                         Special          63
                  

We can see that the Product type column hasn't spilled any further, therefore we only need to combine the Product.1 and type columns. But before that let's filter our data to only include the product types we are interested in. These are Climbing Accessories, Cooking Gear, First Aid, Golf Accessories, Insect Repellents, and Sleeping Bags.

In [30]:
tmp1 = copy.deepcopy(tmp[tmp["Product.1"].isin(["Climbing", "Cooking", "First", "Golf", "Insect", "Sleeping"])])
tmp1.head()

,Year,Product.1,type,Product.2,Order,method,type.1,Retailer,country,Revenue,...,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line
0,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,States,315044,...,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
1,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,...,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
2,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
3,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
4,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,181120,235237,...,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment


In [31]:
tmp1.groupby(["Product.1"])["Year"].count()

Product.1
Climbing    3087
Cooking     5880
First       2205
Golf        1764
Insect      2205
Sleeping    3087
Name: Year, dtype: int64

We can see that tmp1 only contains the product that we are interested in

In [32]:
tmp1.groupby(["Product.1", "type"])["Year"].count()

Product.1  type       
Climbing   Accessories    3087
Cooking    Gear           5880
First      Aid            2205
Golf       Accessories    1764
Insect     Repellents     2205
Sleeping   Bags           3087
Name: Year, dtype: int64

Now we can combine the two columns and obtain the Product type column

In [33]:
tmp1["Product type"] = tmp1["Product.1"] + " " + tmp1["type"]
tmp1 = tmp1.drop(["Product.1", "type"], axis=1)
tmp1.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


We proceed to the next column which is Product which is a little bit completed because there are single and multiple words, spilling into the other columns. We are going to do this procedurally, starting with the ones that have spilled the most.

In [34]:
tmp1.groupby(["Product.2"])["Year"].count()

Product.2
Aloe           441
BugShield     2205
Calamine       441
Compact        441
Course        1764
Deluxe         441
Firefly       1323
Granite       1764
Hibernator    3087
Insect         441
TrailChef     5880
Name: Year, dtype: int64

In [35]:
tmp1.groupby(["Product.2", "Order"])["Year"].count()

Product.2   Order       
Aloe        Relief           441
BugShield   Extreme          441
            Lotion           882
            Natural          441
            Spray            441
Calamine    Relief           441
Compact     Relief           441
Course      Pro             1764
Deluxe      Family           441
Firefly     Charger          441
            Climbing         441
            Rechargeable     441
Granite     Belay            441
            Carabiner        441
            Chalk            441
            Pulley           441
Hibernator  Camp             441
            E-mail            63
            Extreme          441
            Fax               63
            Lite             441
            Mail              63
            Pad              441
            Pillow           441
            Sales             63
            Self             441
            Special           63
            Telephone         63
            Web               63
Insect      Bite  

We can see that there are some products with only one word while others have more than one word.
The ones with one word can be noticed by the following words in the Order column: E-mail, Fax, Mail, Sales, Special, Telephone, Web.
As earlier stated we will start with the longer words that have spilled the farthest to other columns.

In [36]:
tmp2 = copy.deepcopy(tmp1[~tmp1["Order"].isin(["E-mail", "Fax", "Mail", "Sales", "Special", "Telephone", "Web"])])
tmp2.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [37]:
tmp2.groupby(["Product.2", "Order"])["Year"].count()

Product.2   Order       
Aloe        Relief           441
BugShield   Extreme          441
            Lotion           882
            Natural          441
            Spray            441
Calamine    Relief           441
Compact     Relief           441
Course      Pro             1764
Deluxe      Family           441
Firefly     Charger          441
            Climbing         441
            Rechargeable     441
Granite     Belay            441
            Carabiner        441
            Chalk            441
            Pulley           441
Hibernator  Camp             441
            Extreme          441
            Lite             441
            Pad              441
            Pillow           441
            Self             441
Insect      Bite             441
TrailChef   Canteen          588
            Cook             588
            Cup              588
            Deluxe           588
            Double           588
            Kettle           588
            Kitche

Let's check if there are products with more than two words which could have spilled to the method column

In [39]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tmp2.groupby(["Product.2", "Order", "method"])["Year"].count())

Product.2   Order         method   
Aloe        Relief        E-mail        63
                          Fax           63
                          Mail          63
                          Sales         63
                          Special       63
                          Telephone     63
                          Web           63
BugShield   Extreme       E-mail        63
                          Fax           63
                          Mail          63
                          Sales         63
                          Special       63
                          Telephone     63
                          Web           63
            Lotion        E-mail        63
                          Fax           63
                          Lite         441
                          Mail          63
                          Sales         63
                          Special       63
                          Telephone     63
                          Web           63
            Natura

We can see that there are three letter words whose words have spilled to the method column, indicated by words such as bag, kit, flame in the method column. Let's further obtain a subset of these products with three words

In [297]:
tmp3 = copy.deepcopy(tmp2[~tmp2["method"].isin(["E-mail", "Fax", "Mail", "Sales", "Special", "Telephone", "Web"])])
tmp3.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [42]:
tmp3.groupby(["Product.2", "Order", "method"])["Year"].count()

Product.2   Order         method  
BugShield   Lotion        Lite        441
Compact     Relief        Kit         441
Course      Pro           Gloves      441
                          Golf        882
                          Umbrella    441
Deluxe      Family        Relief      441
Firefly     Climbing      Lamp        441
            Rechargeable  Battery     441
Granite     Chalk         Bag         441
Hibernator  Camp          Cot         441
            Self          0           441
Insect      Bite          Relief      441
TrailChef   Cook          Set         588
            Deluxe        Cook        588
            Double        Flame       588
            Kitchen       Kit         588
            Single        Flame       588
            Water         Bag         588
Name: Year, dtype: int64

So far so good, but let's check if there are products with more than three words that could have spilled to the type.1 column

In [44]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tmp3.groupby(["Product.2", "Order", "method", "type.1"])["Year"].count())

Product.2   Order         method    type.1   
BugShield   Lotion        Lite      E-mail        63
                                    Fax           63
                                    Mail          63
                                    Sales         63
                                    Special       63
                                    Telephone     63
                                    Web           63
Compact     Relief        Kit       E-mail        63
                                    Fax           63
                                    Mail          63
                                    Sales         63
                                    Special       63
                                    Telephone     63
                                    Web           63
Course      Pro           Gloves    E-mail        63
                                    Fax           63
                                    Mail          63
                                    Sales         63


Unfortunately there still are as indicated by words such as set, kit and bag in the type.1 column. Let's get a subset with the products with four words

In [298]:
tmp4 = copy.deepcopy(tmp3[~tmp3["type.1"].isin(["E-mail", "Fax", "Mail", "Sales", "Special", "Telephone", "Web"])])
tmp4.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
735,2004,TrailChef,Deluxe,Cook,Set,Telephone,United,States,865351,930482,...,254856.69,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
736,2004,TrailChef,Deluxe,Cook,Set,Telephone,Canada,67799.7,72902.6,47831.8,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
737,2004,TrailChef,Deluxe,Cook,Set,Telephone,Mexico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
738,2004,TrailChef,Deluxe,Cook,Set,Telephone,Brazil,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
739,2004,TrailChef,Deluxe,Cook,Set,Telephone,Japan,419827,451426,296183,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [46]:
tmp4.groupby(["Product.2", "Order", "method", "type.1"])["Year"].count()

Product.2   Order   method  type.1   
Course      Pro     Golf    Bag          441
                            and          441
Deluxe      Family  Relief  Kit          441
Hibernator  Self    0       Inflating    441
TrailChef   Deluxe  Cook    Set          588
Name: Year, dtype: int64

Seems like there products with 5 words let's check, that might have spilled to the Retailer column

In [47]:
tmp4.groupby(["Product.2", "Order", "method", "type.1", "Retailer"])["Year"].count()

Product.2   Order   method  type.1     Retailer 
Course      Pro     Golf    Bag        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
                            and        Tee          441
Deluxe      Family  Relief  Kit        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
Hibernator  Self    0       Inflating  Mat          441
TrailChef   Deluxe  Cook    Set        E-mail        84

Yes, there are. We can see that by seeing values such as Tee and Mat in the Retailer column. Let's get a subset of these products

In [48]:
tmp5 = copy.deepcopy(tmp4[~tmp4["Retailer"].isin(["E-mail", "Fax", "Mail", "Sales", "Special", "Telephone", "Web"])])
tmp5.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2793,2004,Hibernator,Self,0,Inflating,Mat,Telephone,United,States,901517,...,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2794,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Canada,57049.5,61344.1,...,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2795,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Mexico,28879.8,31053.9,...,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2796,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Brazil,19647.8,21126.8,...,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2797,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Japan,423905,470263,...,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [49]:
tmp5.groupby(["Product.2", "Order", "method", "type.1", "Retailer"])["Year"].count()

Product.2   Order  method  type.1     Retailer
Course      Pro    Golf    and        Tee         441
Hibernator  Self   0       Inflating  Mat         441
Name: Year, dtype: int64

To check for products with 6 words that could have spilled to country column

In [53]:
tmp5.groupby(["Product.2", "Order", "method", "type.1", "Retailer", "country"])["Year"].count()

Product.2   Order  method  type.1     Retailer  country  
Course      Pro    Golf    and        Tee       Set          441
Hibernator  Self   0       Inflating  Mat       E-mail        63
                                                Fax           63
                                                Mail          63
                                                Sales         63
                                                Special       63
                                                Telephone     63
                                                Web           63
Name: Year, dtype: int64

We have a product with 6 words(indicated by Set in the column country) therefore we need to get a subset of this too. Huh, quite repititive. But we will endure, hopefully this is the last.

In [54]:
tmp6 = copy.deepcopy(tmp5[~tmp5["country"].isin(["E-mail", "Fax", "Mail", "Sales", "Special", "Telephone", "Web"])])
tmp6.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
17493,2004,Course,Pro,Golf,and,Tee,Set,Telephone,United,States,...,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17494,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Canada,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17495,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Mexico,32224.2,...,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17496,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Brazil,10268.4,...,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17497,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Japan,32314.7,...,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories


In [55]:
tmp6.groupby(["Product.2", "Order", "method", "type.1", "Retailer", "country", "Revenue"])["Year"].count()

Product.2  Order  method  type.1  Retailer  country  Revenue  
Course     Pro    Golf    and     Tee       Set      E-mail       63
                                                     Fax          63
                                                     Mail         63
                                                     Sales        63
                                                     Special      63
                                                     Telephone    63
                                                     Web          63
Name: Year, dtype: int64

Seems that was our last. A product that had 6 words. Taking a look at our data we can see that we can now fix the Product column

In [56]:
tmp6

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
17493,2004,Course,Pro,Golf,and,Tee,Set,Telephone,United,States,...,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17494,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Canada,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17495,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Mexico,32224.2,...,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17496,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Brazil,10268.4,...,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
17497,2004,Course,Pro,Golf,and,Tee,Set,Telephone,Japan,32314.7,...,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59971,2006,Course,Pro,Golf,and,Tee,Set,Fax,Belgium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
59972,2006,Course,Pro,Golf,and,Tee,Set,Fax,Switzerland,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
59973,2006,Course,Pro,Golf,and,Tee,Set,Fax,Austria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories
59974,2006,Course,Pro,Golf,and,Tee,Set,Fax,Italy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories


In [59]:
tmp6["Product"] = tmp6["Product.2"] + " " + tmp6["Order"] + " " + tmp6["method"] + " " + tmp6["type.1"] + " " +                     tmp6["Retailer"] + " " + tmp6["country"]
tmp6 = tmp6.drop(["Product.2", "Order", "method", "type.1", "Retailer", "country"], axis=1)
tmp6.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17493,2004,Telephone,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17494,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17495,2004,Telephone,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17496,2004,Telephone,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17497,2004,Telephone,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


Let's try and fix the other columns for this product with six words

In [60]:
tmp6.groupby(["Revenue"])["Year"].count()

Revenue
E-mail       63
Fax          63
Mail         63
Sales        63
Special      63
Telephone    63
Web          63
Name: Year, dtype: int64

The Revenue column is supposedly the order method type column, but let's check if there is any order method that has more than two words

In [62]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tmp6.groupby(["Revenue", "Planned"])["Year"].count())

Revenue    Planned    
E-mail     Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico          3
           Netherlands     3
           Singapore       3
           Spain           3
           Sweden          3
           Switzerland     3
           United          6
Fax        Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico   

We can see that there's a order method type called sales visit whose values have spilled to the planned. Let's get a subset of this.

In [64]:
tmp7 = copy.deepcopy(tmp6[tmp6["Planned"] == "visit"])
tmp7.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17514,2004,Sales,visit,United,States,5819.70,6586.16,1733.2,619.0,2.80,10.64,4086.500000,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17515,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17516,2004,Sales,visit,Mexico,13497.6,14598.08,3841.60,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17517,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17518,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [65]:
tmp7.groupby(["Revenue", "Planned", "revenue"])["Year"].count()

Revenue  Planned  revenue    
Sales    visit    Australia      3
                  Austria        3
                  Belgium        3
                  Brazil         3
                  Canada         3
                  China          3
                  Denmark        3
                  Finland        3
                  France         3
                  Germany        3
                  Italy          3
                  Japan          3
                  Korea          3
                  Mexico         3
                  Netherlands    3
                  Singapore      3
                  Spain          3
                  Sweden         3
                  Switzerland    3
                  United         6
Name: Year, dtype: int64

Now we can fix this

In [66]:
tmp7

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17514,2004,Sales,visit,United,States,5819.70,6586.16,1733.2,619.0,2.80,10.64,4086.500000,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17515,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17516,2004,Sales,visit,Mexico,13497.6,14598.08,3841.60,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17517,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17518,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59866,2006,Sales,visit,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
59867,2006,Sales,visit,Switzerland,28451.8,30600.64,8052.80,2876.0,2.8,10.64,20398.98,5.816667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
59868,2006,Sales,visit,Austria,0,425.60,112.00,40.0,2.8,10.64,-112.00,0.000000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
59869,2006,Sales,visit,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [67]:
tmp7["Order method type"] = tmp7["Revenue"] + " " + tmp7["Planned"]
tmp7 = tmp7.drop(["Revenue", "Planned"], axis=1)
tmp7.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17514,2004,United,States,5819.70,6586.16,1733.2,619.0,2.80,10.64,4086.500000,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17515,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17516,2004,Mexico,13497.6,14598.08,3841.60,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17517,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17518,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


Let's look at the next column, which supposedly is Retailer country

In [70]:
tmp7.groupby(["revenue", "Product.3"])["Year"].count()

revenue      Product.3
Australia    21183.33     1
             24114.16     1
Austria      0            3
China        27162.24     1
Denmark      6013.69      1
Finland      21185.75     1
France       0            1
             16307.46     1
             20542.05     1
Germany      3962.88      1
Italy        0            1
Japan        16448.11     1
Korea        8433.46      1
Mexico       13497.62     1
             23901.12     1
             30444        1
Netherlands  10970.16     1
Singapore    7422.67      1
Sweden       13058.36     1
Switzerland  12745.46     1
             28451.78     1
United       Kingdom      3
             States       3
Name: Year, dtype: int64

We can see that some countries(United Kingdom, United States) have more than two word that spilled to the next column. Let's get a subset of this data.

In [77]:
tmp8 = copy.deepcopy(tmp7[tmp7["Product.3"].isin(["Kingdom", "States"])])
tmp8.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17514,2004,United,States,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17529,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
38682,2005,United,States,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
38697,2005,United,Kingdom,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
59850,2006,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


In [78]:
tmp8.groupby(["revenue", "Product.3", "cost"])["Year"].count()

revenue  Product.3  cost    
United   Kingdom    8750.77     1
                    27987.84    1
         States     5819.70     1
                    10904.28    1
Name: Year, dtype: int64

In [79]:
tmp8["Retailer country"] = tmp8["revenue"] + " " + tmp8["Product.3"]
tmp8 = tmp8.drop(["revenue", "Product.3"], axis=1)
tmp8.head()

,Year,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
17514,2004,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States
17529,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom
38682,2005,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States
38697,2005,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom
59850,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States


The rest of the columns can simply be translated and dropping the respective column not needed

In [80]:
tmp8["Revenue"] = tmp8["cost"]
tmp8 = tmp8.drop(["cost"], axis=1)

tmp8["Planned revenue"] = tmp8["Quantity"]
tmp8 = tmp8.drop(["Quantity"], axis=1)

tmp8["Product cost"] = tmp8["Unit"]
tmp8 = tmp8.drop(["Unit"], axis=1)

tmp8["Quantity"] = tmp8["cost.1"]
tmp8 = tmp8.drop(["cost.1"], axis=1)

tmp8["Unit cost"] = tmp8["Unit.1"]
tmp8 = tmp8.drop(["Unit.1"], axis=1)

tmp8["Unit price"] = tmp8["price"]
tmp8 = tmp8.drop(["price"], axis=1)

tmp8["Gross profit"] = tmp8["Gross"]
tmp8 = tmp8.drop(["Gross"], axis=1)

tmp8["Unit sale price"] = tmp8["profit"]
tmp8 = tmp8.drop(["profit", "Unit.2", "sale", "price.1" ], axis=1)

tmp8

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17514,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,5819.70,6586.16,1733.2,619.0,2.8,10.64,4086.50,5.105
17529,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38682,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,10904.28,11363.52,2990.4,1068.0,2.8,10.64,7913.88,10.210
38697,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,27987.84,28855.68,7593.6,2712.0,2.8,10.64,20394.24,10.320
59850,2006,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59865,2006,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,8750.77,8926.96,2349.2,839.0,2.8,10.64,6401.57,10.430


Now this is structured like we want it to be so let us store it

In [81]:
str1 = tmp8

Our last stop over was countries with more than two words. With our subsets now we can walk backwards. For example try to fix columns for countries with just one word. We first start by creating a subset that has countries with just one word

In [84]:
temp = copy.deepcopy(tmp7[~tmp7.index.isin(tmp8.index.values)])
temp.head()

,Year,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17515,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17516,2004,Mexico,13497.6,14598.08,3841.6,1372.0,2.8,10.64,9656.02,6.806667,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17517,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17518,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit
17519,2004,Korea,8433.46,8788.64,2312.8,826.0,2.8,10.64,6120.66,10.210000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit


In [85]:
temp.groupby(["revenue", "Product.3"])["Year"].count()

revenue      Product.3
Australia    21183.33     1
             24114.16     1
Austria      0.00         3
China        27162.24     1
Denmark      6013.69      1
Finland      21185.75     1
France       0.00         1
             16307.46     1
             20542.05     1
Germany      3962.88      1
Italy        0.00         1
Japan        16448.11     1
Korea        8433.46      1
Mexico       13497.62     1
             23901.12     1
             30444.00     1
Netherlands  10970.16     1
Singapore    7422.67      1
Sweden       13058.36     1
Switzerland  12745.46     1
             28451.78     1
Name: Year, dtype: int64

Looks we can just translate the rest

In [86]:
temp["Retailer country"] = temp["revenue"]
temp = temp.drop(["revenue"], axis=1)

temp["Revenue"] = temp["Product.3"]
temp = temp.drop(["Product.3"], axis=1)

temp["Planned revenue"] = temp["cost"]
temp = temp.drop(["cost"], axis=1)

temp["Product cost"] = temp["Quantity"]
temp = temp.drop(["Quantity"], axis=1)

temp["Quantity"] = temp["Unit"]
temp = temp.drop(["Unit"], axis=1)

temp["Unit cost"] = temp["cost.1"]
temp = temp.drop(["cost.1"], axis=1)

temp["Unit price"] = temp["Unit.1"]
temp = temp.drop(["Unit.1"], axis=1)

temp["Gross profit"] = temp["price"]
temp = temp.drop(["price"], axis=1)

temp["Unit sale price"] = temp["Gross"]
temp = temp.drop(["Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17515,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17516,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Mexico,13497.6,14598.08,3841.6,1372.0,2.8,10.64,9656.02,6.806667
17517,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17518,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17519,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,Korea,8433.46,8788.64,2312.8,826.0,2.8,10.64,6120.66,10.210000


This will be our second structured data

In [87]:
str2 = temp

We are done with fixing the Retailer country. We look at the Order method type where our last fix at tmp6 we only looked at Order method  types with more than one word(Sales visit). So we get the subset with one word and fix it.

In [89]:
tmp6.groupby(["Revenue"])["Year"].count()

Revenue
E-mail       63
Fax          63
Mail         63
Sales        63
Special      63
Telephone    63
Web          63
Name: Year, dtype: int64

We want a subset that doesn't contain the sales value

In [212]:
temp = copy.deepcopy(tmp6[~tmp6.index.isin(tmp7.index.values)])
temp.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
17493,2004,Telephone,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17494,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17495,2004,Telephone,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17496,2004,Telephone,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set
17497,2004,Telephone,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set


In [213]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["Revenue", "Planned"])["Year"].count())

Revenue    Planned    
E-mail     Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands    3
           Singapore      3
           Spain          3
           Sweden         3
           Switzerland    3
           United         6
Fax        Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands   

This is easily fixed by translation

In [214]:
temp["Order method type"] = temp["Revenue"]
temp = temp.drop(["Revenue"], axis=1)
temp.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17493,2004,United,States,159493,171697.68,45183.6,16137.0,2.80,10.64,114309.370000,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17494,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17495,2004,Mexico,32224.2,35005.6,9212.00,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17496,2004,Brazil,10268.4,12012.6,3161.20,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17497,2004,Japan,32314.7,35271.6,9282.00,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone


In [215]:
temp.groupby(["Planned", "revenue"])["Year"].count()

Planned      revenue 
Australia    14351.68     1
             18398.52     1
             72206.89     1
Austria      12105.36     1
             12414.96     1
                         ..
Switzerland  11356.35     1
             25314.3      1
             33003.92     1
United       Kingdom     18
             States      18
Name: Year, Length: 118, dtype: int64

So now we need to fix Retailer country with more than two words then fix it for those Retailer coutries with just one word

In [216]:
temp1 = copy.deepcopy(temp[temp["revenue"].isin(["Kingdom", "States"])])
temp1.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17493,2004,United,States,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17508,2004,United,Kingdom,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17535,2004,United,States,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web
17550,2004,United,Kingdom,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web
17556,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special


In [217]:
temp1.groupby(["Planned", "revenue"])["Year"].count()

Planned  revenue
United   Kingdom    18
         States     18
Name: Year, dtype: int64

In [218]:
temp1["Retailer country"] = temp1["Planned"] + " " + temp1["revenue"]
temp1= temp1.drop(["Planned", "revenue"], axis=1)
temp1.head()

,Year,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type,Retailer country
17493,2004,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United States
17508,2004,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United Kingdom
17535,2004,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United States
17550,2004,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United Kingdom
17556,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special,United States


Fix the rest by translation

In [219]:
temp1["Revenue"] = temp1["Product.3"]
temp1 = temp1.drop(["Product.3"], axis=1)

temp1["Planned revenue"] = temp1["cost"]
temp1 = temp1.drop(["cost"], axis=1)

temp1["Product cost"] = temp1["Quantity"]
temp1 = temp1.drop(["Quantity"], axis=1)

temp1["Quantity"] = temp1["Unit"]
temp1 = temp1.drop(["Unit"], axis=1)

temp1["Unit cost"] = temp1["cost.1"]
temp1 = temp1.drop(["cost.1"], axis=1)

temp1["Unit price"] = temp1["Unit.1"]
temp1 = temp1.drop(["Unit.1"], axis=1)

temp1["Gross profit"] = temp1["price"]
temp1 = temp1.drop(["price"], axis=1)

temp1["Unit sale price"] = temp1["Gross"]
temp1 = temp1.drop(["Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp1.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17493,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United States,159493,171697.68,45183.6,16137.0,2.8,10.64,114309.37,7.891613
17508,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,United Kingdom,66286.3,72362.64,19042.8,6801.0,2.8,10.64,47243.46,6.950000
17535,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United States,159041,167824.72,44164.4,15773.0,2.8,10.64,114876.32,9.286818
17550,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Web,United Kingdom,59938,62318.48,16399.6,5857.0,2.8,10.64,43538.37,10.240909
17556,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Well structured, let's save this.

In [220]:
str3 = temp1

We need to solve for countries that just have one word now

In [221]:
temp2 = copy.deepcopy(temp[~temp.index.isin(temp1.index.values)])
temp2.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
17494,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17495,2004,Mexico,32224.2,35005.6,9212.0,3290.0,2.8,10.64,23012.18,7.308571,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17496,2004,Brazil,10268.4,12012.6,3161.2,1129.0,2.8,10.64,7107.20,4.128000,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17497,2004,Japan,32314.7,35271.6,9282.0,3315.0,2.8,10.64,23032.65,7.292857,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone
17498,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone


In [222]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["Planned", "revenue"])["Year"].count())

Planned      revenue  
Australia    14351.68     1
             18398.52     1
             72206.89     1
Austria      12105.36     1
             12414.96     1
             28380.00     1
             53965.97     1
             79007.25     1
Belgium      3560.40      1
             17956.80     1
             69039.45     1
             69272.28     1
Brazil       10268.40     1
             46289.46     1
             66488.85     1
             74829.65     1
Canada       5697.18      1
             6759.60      1
             6789.65      1
             10313.96     1
             34408.13     1
             46460.46     1
             78873.89     1
             83518.71     1
             187139.46    1
China        0.00         1
             10686.27     1
             36939.78     1
             40829.79     1
             48973.85     1
             54872.68     1
             62342.26     1
             66278.36     1
             110095.19    1
Denmark      3849.17     

In [223]:
temp2["Retailer country"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Planned revenue"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Product cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit cost"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit price"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Gross profit"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1"], axis=1)

temp2["Unit sale price"] = temp2["price"]
temp2 = temp2.drop(["price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Quantity,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Gross profit,Unit sale price
17494,2004,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17495,2004,3290.0,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Mexico,32224.2,35005.6,9212.0,2.8,10.64,23012.18,7.308571
17496,2004,1129.0,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Brazil,10268.4,12012.6,3161.2,2.8,10.64,7107.20,4.128000
17497,2004,3315.0,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Japan,32314.7,35271.6,9282.0,2.8,10.64,23032.65,7.292857
17498,2004,NaN,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17494,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17495,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Mexico,32224.2,35005.6,9212.0,3290.0,2.8,10.64,23012.18,7.308571
17496,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Brazil,10268.4,12012.6,3161.2,1129.0,2.8,10.64,7107.20,4.128000
17497,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Japan,32314.7,35271.6,9282.0,3315.0,2.8,10.64,23032.65,7.292857
17498,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
str4 = temp2

We now proceed to the second longest product and fix it

In [226]:
temp = copy.deepcopy(tmp5[~tmp5.index.isin(tmp6.index.values)])
temp.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2793,2004,Hibernator,Self,0,Inflating,Mat,Telephone,United,States,901517,...,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2794,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Canada,57049.5,61344.1,...,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2795,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Mexico,28879.8,31053.9,...,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2796,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Brazil,19647.8,21126.8,...,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2797,2004,Hibernator,Self,0,Inflating,Mat,Telephone,Japan,423905,470263,...,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [227]:
temp.groupby(["Product.2", "Order", "method", "type.1", "Retailer", "country"])["Year"].count()

Product.2   Order  method  type.1     Retailer  country  
Hibernator  Self   0       Inflating  Mat       E-mail       63
                                                Fax          63
                                                Mail         63
                                                Sales        63
                                                Special      63
                                                Telephone    63
                                                Web          63
Name: Year, dtype: int64

Let's find a way of replacing the 0 value in the method column with a hyphen so that it can read "self-inflating"

In [228]:
def check(k):
    if k==0:
        return "-"
    else:
        return k

In [229]:
temp["Product"] = temp["Product.2"] + " " + temp["Order"] + " " + temp["method"].map(check) + " " + temp["type.1"] + " " + temp["Retailer"]
temp = temp.drop(["Product.2", "Order", "method", "type.1", "Retailer"], axis=1)
temp.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2793,2004,Telephone,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2794,2004,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2795,2004,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2796,2004,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2797,2004,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat


Remember to solve the order method type we need to solve for the two-word and one-word method types and for retailer country we need to solve for two-word and one-word countries 

In [230]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["country", "Revenue"])["Year"].count())

country    Revenue    
E-mail     Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico          3
           Netherlands     3
           Singapore       3
           Spain           3
           Sweden          3
           Switzerland     3
           United          6
Fax        Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico   

In [231]:
temp1 = copy.deepcopy(temp[temp["Revenue"] == "visit"])
temp1.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2814,2004,Sales,visit,United,States,339693,365264.9,198030.0,2870.0,69.00,127.27,141663.200000,118.36,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2815,2004,Sales,visit,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2816,2004,Sales,visit,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2817,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2818,2004,Sales,visit,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat


In [232]:
print(temp1.groupby(["country", "Revenue"])["Year"].count())

country  Revenue
Sales    visit      63
Name: Year, dtype: int64


In [233]:
temp1["Order method type"] = temp1["country"] + " " + temp1["Revenue"]
temp1 = temp1.drop(["country", "Revenue"], axis=1)
temp1.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2814,2004,United,States,339693,365264.9,198030.0,2870.0,69.00,127.27,141663.200000,118.36,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2815,2004,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2816,2004,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2817,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2818,2004,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit


Solving for countries with more than two words

In [234]:
temp1.groupby(["Planned", "revenue"])["Year"].count()

Planned      revenue  
Australia    92254.33     1
             131545.96    1
Austria      11072.2      1
             96517.13     1
Belgium      17517.28     1
             117835.55    1
Canada       22370.04     1
             27275.64     1
             134331.01    1
China        62327.23     1
             355233.58    1
Denmark      30418.52     1
             31436.6      1
             71179.85     1
Finland      42827.54     1
             139138.96    1
France       77879.13     1
             104103.51    1
             195817.7     1
Germany      146546.75    1
             199791.68    1
Italy        66035.76     1
             196018.75    1
Japan        32193.92     1
Korea        185426.5     1
             355383.79    1
Mexico       81251.52     1
             240870.79    1
             303740.57    1
Netherlands  32060.84     1
             152867.28    1
Singapore    201023.15    1
Spain        24145.44     1
Switzerland  18423.02     1
             138804.68   

In [235]:
temp2 = copy.deepcopy(temp1[temp1["revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2814,2004,United,States,339693,365264.90,198030.0,2870.0,69.00,127.27,141663.20,118.360,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2829,2004,United,Kingdom,22961.8,24690.38,13386.0,194.0,69.00,127.27,9575.84,118.360,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
23982,2005,United,States,206481,219668.02,94153.3,1726.0,54.55,127.27,112328.08,119.630,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
23997,2005,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
45150,2006,United,States,236268,264212.52,109093.8,2076.0,52.55,127.27,127174.02,115.818,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit


In [236]:
temp2.groupby(["Planned", "revenue", "Product.3"])["Year"].count()

Planned  revenue  Product.3
United   Kingdom  22961.84     1
                  42197.59     1
         States   206481.38    1
                  236267.82    1
                  339693.20    1
Name: Year, dtype: int64

In [237]:
temp2["Retailer country"] = temp2["Planned"] + " " + temp2["revenue"]
temp2 = temp2.drop(["Planned", "revenue"], axis=1)

temp2["Revenue"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Planned revenue"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Product cost"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit cost"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Unit price"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1"], axis=1)

temp2["Gross profit"] = temp2["price"]
temp2 = temp2.drop(["price"], axis=1)

temp2["Unit sale price"] = temp2["Gross"]
temp2 = temp2.drop(["Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2814,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,United States,339693,365264.90,198030.0,2870.0,69.00,127.27,141663.20,118.360
2829,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,United Kingdom,22961.8,24690.38,13386.0,194.0,69.00,127.27,9575.84,118.360
23982,2005,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,United States,206481,219668.02,94153.3,1726.0,54.55,127.27,112328.08,119.630
23997,2005,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45150,2006,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,United States,236268,264212.52,109093.8,2076.0,52.55,127.27,127174.02,115.818


In [238]:
str5 = temp2

In [239]:
temp2 = copy.deepcopy(temp1[~temp1["revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2815,2004,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2816,2004,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2817,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2818,2004,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit
2819,2004,Korea,355384,397464,215487.0,3123.0,69.0,127.27,139896.79,114.701250,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit


In [240]:
temp2.groupby(["Planned", "revenue"])["Year"].count()

Planned      revenue  
Australia    92254.33     1
             131545.96    1
Austria      11072.20     1
             96517.13     1
Belgium      17517.28     1
             117835.55    1
Canada       22370.04     1
             27275.64     1
             134331.01    1
China        62327.23     1
             355233.58    1
Denmark      30418.52     1
             31436.60     1
             71179.85     1
Finland      42827.54     1
             139138.96    1
France       77879.13     1
             104103.51    1
             195817.70    1
Germany      146546.75    1
             199791.68    1
Italy        66035.76     1
             196018.75    1
Japan        32193.92     1
Korea        185426.50    1
             355383.79    1
Mexico       81251.52     1
             240870.79    1
             303740.57    1
Netherlands  32060.84     1
             152867.28    1
Singapore    201023.15    1
Spain        24145.44     1
Switzerland  18423.02     1
             138804.68   

In [241]:
temp2["Retailer country"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Planned revenue"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Product cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit cost"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit price"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Gross profit"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1"], axis=1)

temp2["Unit sale price"] = temp2["price"]
temp2 = temp2.drop(["price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Quantity,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Gross profit,Unit sale price
2815,2004,189.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Canada,22370,24054,13041.0,69.0,127.27,9329.04,118.360000
2816,2004,2110.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Mexico,240871,268540,145590.0,69.0,127.27,95280.79,115.107778
2817,2004,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2818,2004,272.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Japan,32193.9,34617.4,18768.0,69.0,127.27,13425.92,118.360000
2819,2004,3123.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Korea,355384,397464,215487.0,69.0,127.27,139896.79,114.701250


In [242]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2815,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Canada,22370,24054,13041.0,189.0,69.0,127.27,9329.04,118.360000
2816,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Mexico,240871,268540,145590.0,2110.0,69.0,127.27,95280.79,115.107778
2817,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2818,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Japan,32193.9,34617.4,18768.0,272.0,69.0,127.27,13425.92,118.360000
2819,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Sales visit,Korea,355384,397464,215487.0,3123.0,69.0,127.27,139896.79,114.701250


In [243]:
str6 = temp2

In [244]:
temp1 = copy.deepcopy(temp[temp["Revenue"] != "visit"])
temp1.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2793,2004,Telephone,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2794,2004,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2795,2004,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2796,2004,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat
2797,2004,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat


In [245]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["country", "Revenue"])["Year"].count())

country    Revenue    
E-mail     Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands    3
           Singapore      3
           Spain          3
           Sweden         3
           Switzerland    3
           United         6
Fax        Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands   

In [246]:
temp1["Order method type"] = temp1["country"]
temp1 = temp1.drop(["country"], axis=1)
temp1.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2793,2004,United,States,901517,1.06232e+06,575943.0,8347.0,69.00,127.27,325573.610000,110.1644,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2794,2004,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2795,2004,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2796,2004,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2797,2004,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone


In [247]:
temp1.groupby(["Revenue", "Planned"])["Year"].count()

Revenue      Planned  
Australia    47880.36      1
             63882.42      1
             96997.07      1
             412686.39     1
Austria      60680.79      1
                          ..
Sweden       172236.54     1
Switzerland  186830.31     1
             219900.24     1
United       Kingdom      18
             States       18
Name: Year, Length: 126, dtype: int64

In [248]:
temp2 = copy.deepcopy(temp1[temp1["Planned"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2793,2004,United,States,901517,1.06232e+06,575943.0,8347.0,69.0,127.27,325573.61,110.164400,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2808,2004,United,Kingdom,297037,352665,191199.0,2771.0,69.0,127.27,105838.17,109.579000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2835,2004,United,States,411303,457027,247779.0,3591.0,69.0,127.27,163524.13,115.920833,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web
2850,2004,United,Kingdom,317259,349356,189405.0,2745.0,69.0,127.27,127853.73,116.108462,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web
2856,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Special


In [249]:
temp2.groupby(["Revenue", "Planned", "revenue"])["Year"].count()

Revenue  Planned  revenue   
United   Kingdom  60173.89      1
                  297037.17     1
                  317258.73     1
                  639525.72     1
                  783749.89     1
         States   38760.12      1
                  39295.52      1
                  98335.86      1
                  411303.13     1
                  439524.68     1
                  901516.61     1
                  1147995.48    1
                  2157500.04    1
Name: Year, dtype: int64

In [250]:
temp2["Retailer country"] = temp2["Revenue"] + " " + temp2["Planned"]
temp2 = temp2.drop(["Revenue", "Planned"], axis=1)

temp2["Revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Planned revenue"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Product cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit cost"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit price"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Gross profit"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1"], axis=1)

temp2["Unit sale price"] = temp2["price"]
temp2 = temp2.drop(["price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Quantity,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Gross profit,Unit sale price
2793,2004,8347.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,United States,901517,1.06232e+06,575943.0,69.0,127.27,325573.61,110.164400
2808,2004,2771.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,United Kingdom,297037,352665,191199.0,69.0,127.27,105838.17,109.579000
2835,2004,3591.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web,United States,411303,457027,247779.0,69.0,127.27,163524.13,115.920833
2850,2004,2745.0,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web,United Kingdom,317259,349356,189405.0,69.0,127.27,127853.73,116.108462
2856,2004,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2793,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,United States,901517,1.06232e+06,575943.0,8347.0,69.0,127.27,325573.61,110.164400
2808,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,United Kingdom,297037,352665,191199.0,2771.0,69.0,127.27,105838.17,109.579000
2835,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web,United States,411303,457027,247779.0,3591.0,69.0,127.27,163524.13,115.920833
2850,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Web,United Kingdom,317259,349356,189405.0,2745.0,69.0,127.27,127853.73,116.108462
2856,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
str7 = temp2

In [253]:
temp2 = copy.deepcopy(temp1[~temp1["Planned"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2794,2004,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2795,2004,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2796,2004,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2797,2004,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone
2798,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone


In [254]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["Revenue", "Planned"])["Year"].count())

Revenue      Planned  
Australia    47880.36     1
             63882.42     1
             96997.07     1
             412686.39    1
Austria      60680.79     1
             126807.80    1
             137259.88    1
             182645.80    1
             336946.91    1
Belgium      10415.68     1
             10889.12     1
             49331.28     1
             80444.52     1
             175778.58    1
             337863.57    1
Brazil       17872.36     1
             19647.76     1
             219399.67    1
             321896.44    1
             450531.42    1
Canada       25241.93     1
             27993.42     1
             29234.92     1
             36635.73     1
             57049.52     1
             121030.91    1
             243510.38    1
             292556.05    1
             641166.05    1
             699937.20    1
China        61547.20     1
             62967.52     1
             161979.02    1
             551924.81    1
             605489.13   

In [255]:
temp2["Retailer country"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Planned revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Product cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Unit price"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Quantity"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Gross profit"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Unit sale price"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1", "price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Quantity,Gross profit,Unit sale price
2794,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Canada,57049.5,61344.1,33258,69.0,127.27,482.0,23791.52,118.360000
2795,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Mexico,28879.8,31053.9,16836,69.0,127.27,244.0,12043.84,118.360000
2796,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Brazil,19647.8,21126.8,11454,69.0,127.27,166.0,8193.76,118.360000
2797,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Japan,423905,470263,254955,69.0,127.27,3695.0,168950.27,115.920833
2798,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2794,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Canada,57049.5,61344.1,33258,482.0,69.0,127.27,23791.52,118.360000
2795,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Mexico,28879.8,31053.9,16836,244.0,69.0,127.27,12043.84,118.360000
2796,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Brazil,19647.8,21126.8,11454,166.0,69.0,127.27,8193.76,118.360000
2797,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Japan,423905,470263,254955,3695.0,69.0,127.27,168950.27,115.920833
2798,2004,Camping Equipment,Sleeping Bags,Hibernator Self - Inflating Mat,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
str8 = temp2

Let's handle products with four words

In [258]:
tmp4.groupby(["Product.2", "Order", "method", "type.1", "Retailer"])["Year"].count()

Product.2   Order   method  type.1     Retailer 
Course      Pro     Golf    Bag        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
                            and        Tee          441
Deluxe      Family  Relief  Kit        E-mail        63
                                       Fax           63
                                       Mail          63
                                       Sales         63
                                       Special       63
                                       Telephone     63
                                       Web           63
Hibernator  Self    0       Inflating  Mat          441
TrailChef   Deluxe  Cook    Set        E-mail        84

In [259]:
temp = copy.deepcopy(tmp4[~tmp4.index.isin(tmp5.index.values)])
temp.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
735,2004,TrailChef,Deluxe,Cook,Set,Telephone,United,States,865351,930482,...,254856.69,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
736,2004,TrailChef,Deluxe,Cook,Set,Telephone,Canada,67799.7,72902.6,47831.8,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
737,2004,TrailChef,Deluxe,Cook,Set,Telephone,Mexico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
738,2004,TrailChef,Deluxe,Cook,Set,Telephone,Brazil,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
739,2004,TrailChef,Deluxe,Cook,Set,Telephone,Japan,419827,451426,296183,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [260]:
temp.groupby(["Product.2", "Order", "method", "type.1", "Retailer"])["Year"].count()

Product.2  Order   method  type.1  Retailer 
Course     Pro     Golf    Bag     E-mail       63
                                   Fax          63
                                   Mail         63
                                   Sales        63
                                   Special      63
                                   Telephone    63
                                   Web          63
Deluxe     Family  Relief  Kit     E-mail       63
                                   Fax          63
                                   Mail         63
                                   Sales        63
                                   Special      63
                                   Telephone    63
                                   Web          63
TrailChef  Deluxe  Cook    Set     E-mail       84
                                   Fax          84
                                   Mail         84
                                   Sales        84
                                   Sp

In [261]:
temp["Product"] = temp["Product.2"] + " " + temp["Order"] + " " + temp["method"] + " " + temp["type.1"]
temp = temp.drop(["Product.2", "Order", "method", "type.1"], axis=1)
temp.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
735,2004,Telephone,United,States,865351,930482,610494,7173.00,85.11,129.72,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
736,2004,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
737,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
738,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
739,2004,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set


In [262]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["Retailer", "country"])["Year"].count())

Retailer   country    
E-mail     Australia       10
           Austria         10
           Belgium         10
           Brazil          10
           Canada          10
           China           10
           Denmark         10
           Finland         10
           France          10
           Germany         10
           Italy           10
           Japan           10
           Korea           10
           Mexico          10
           Netherlands     10
           Singapore       10
           Spain           10
           Sweden          10
           Switzerland     10
           United          20
Fax        Australia       10
           Austria         10
           Belgium         10
           Brazil          10
           Canada          10
           China           10
           Denmark         10
           Finland         10
           France          10
           Germany         10
           Italy           10
           Japan           10
           Korea 

In [263]:
temp1 = copy.deepcopy(temp[temp["country"] == "visit"])
temp1.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
756,2004,Sales,visit,United,States,465791,500849,328609.71,3861.00,85.11,...,137181.33,120.64,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
757,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
758,2004,Sales,visit,Mexico,365177,392662,257628,3027.00,85.11,129.72,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
759,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
760,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set


In [264]:
temp1.groupby(["Retailer", "country", "Revenue"])["Year"].count()

Retailer  country  Revenue    
Sales     visit    Australia      10
                   Austria        10
                   Belgium        10
                   Brazil         10
                   Canada         10
                   China          10
                   Denmark        10
                   Finland        10
                   France         10
                   Germany        10
                   Italy          10
                   Japan          10
                   Korea          10
                   Mexico         10
                   Netherlands    10
                   Singapore      10
                   Spain          10
                   Sweden         10
                   Switzerland    10
                   United         20
Name: Year, dtype: int64

In [265]:
temp1["Order method type"] = temp1["Retailer"] + " " + temp1["country"]
temp1 = temp1.drop(["Retailer", "country"], axis=1)
temp1.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
756,2004,United,States,465791,500849,328609.71,3861.00,85.11,129.72,137181.33,120.64,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
757,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
758,2004,Mexico,365177,392662,257628,3027.00,85.11,129.72,107549.31,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
759,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
760,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit


In [266]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["Revenue", "Planned", "revenue"])["Year"].count())

Revenue      Planned    revenue  
Australia    2503.9     2555.00      1
             26507.25   30555.00     1
             60182.16   64254.90     1
             66790.66   70308.24     1
             76306.67   79167.30     1
             133036.54  141524.52    1
Austria      2105.28    2631.60      1
             3207.75    4935.00      1
             5670       5670.00      1
             8772       8772.00      1
             11842.2    13158.00     1
             43376.96   45661.44     1
             54141.36   57595.68     1
             132841.94  139838.16    1
Belgium      6685.35    7175.00      1
             22318.4    23998.20     1
             29451.97   31003.08     1
             79748.76   89247.36     1
Canada       2625       2625.00      1
             22292.38   23465.10     1
             29265.6    31132.80     1
             175849.21  185110.44    1
China        9896.25    15225.00     1
             54387.4    56579.40     1
             106963.64  112596

In [267]:
temp2 = copy.deepcopy(temp1[temp1["Planned"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
756,2004,United,States,465791,500849,328609.71,3861.0,85.11,129.72,137181.33,120.640000,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
771,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
15309,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit
15324,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit
17808,2004,United,States,40004.9,44517.9,16179.10,203.0,79.70,219.30,23825.76,194.446667,NaN,NaN,NaN,NaN,NaN,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit


In [268]:
temp2.groupby(["Revenue", "Planned", "revenue"])["Year"].count()

Revenue  Planned  revenue  
United   Kingdom  0.00         1
                  2170.00      1
                  59986.74     1
                  86630.69     1
         States   6210.75      1
                  6947.49      1
                  7910.00      1
                  19583.30     1
                  40004.86     1
                  107703.02    1
                  193840.79    1
                  368380.74    1
                  465791.04    1
Name: Year, dtype: int64

In [269]:
temp2["Retailer country"] = temp2["Revenue"] + " " + temp2["Planned"]
temp2 = temp2.drop(["Revenue", "Planned"], axis=1)

temp2["Revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Planned revenue"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Product cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit cost"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit price"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Gross profit"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1"], axis=1)

temp2["Unit sale price"] = temp2["price"]
temp2 = temp2.drop(["price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Quantity,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Gross profit,Unit sale price
756,2004,3861.0,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United States,465791,500849,328609.71,85.11,129.72,137181.33,120.640000
771,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15309,2004,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15324,2004,NaN,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17808,2004,203.0,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit,United States,40004.9,44517.9,16179.10,79.70,219.30,23825.76,194.446667


In [270]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
756,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United States,465791,500849,328609.71,3861.0,85.11,129.72,137181.33,120.640000
771,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15309,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15324,2004,Outdoor Protection,First Aid,Deluxe Family Relief Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17808,2004,Golf Equipment,Golf Accessories,Course Pro Golf Bag,Sales visit,United States,40004.9,44517.9,16179.10,203.0,79.70,219.30,23825.76,194.446667


In [271]:
str9 = temp2

In [272]:
temp2 = copy.deepcopy(temp1[~temp1["Planned"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
757,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
758,2004,Mexico,365177,392662,257628,3027.0,85.11,129.72,107549.31,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
759,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
760,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit
761,2004,Korea,522854,562206,368867,4334.0,85.11,129.72,153987.02,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit


In [273]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["Revenue", "Planned", "revenue"])["Year"].count())

Revenue      Planned    revenue  
Australia    2503.90    2555.00      1
             26507.25   30555.00     1
             60182.16   64254.90     1
             66790.66   70308.24     1
             76306.67   79167.30     1
             133036.54  141524.52    1
Austria      2105.28    2631.60      1
             3207.75    4935.00      1
             5670.00    5670.00      1
             8772.00    8772.00      1
             11842.20   13158.00     1
             43376.96   45661.44     1
             54141.36   57595.68     1
             132841.94  139838.16    1
Belgium      6685.35    7175.00      1
             22318.40   23998.20     1
             29451.97   31003.08     1
             79748.76   89247.36     1
Canada       2625.00    2625.00      1
             22292.38   23465.10     1
             29265.60   31132.80     1
             175849.21  185110.44    1
China        9896.25    15225.00     1
             54387.40   56579.40     1
             106963.64  112596

In [274]:
temp2["Retailer country"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Planned revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Product cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Unit price"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Quantity"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Gross profit"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Unit sale price"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1", "price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Quantity,Gross profit,Unit sale price
757,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Mexico,365177,392662,257628,85.11,129.72,3027.0,107549.31,120.64
759,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Korea,522854,562206,368867,85.11,129.72,4334.0,153987.02,120.64


In [275]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
757,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Mexico,365177,392662,257628,3027.0,85.11,129.72,107549.31,120.64
759,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Sales visit,Korea,522854,562206,368867,4334.0,85.11,129.72,153987.02,120.64


In [276]:
str10 = temp2

In [277]:
temp1 = copy.deepcopy(temp[temp["country"] != "visit"])
temp1.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
735,2004,Telephone,United,States,865351,930482,610494,7173.00,85.11,129.72,...,120.64,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
736,2004,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
737,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
738,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set
739,2004,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set


In [278]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["Retailer", "country"])["Year"].count())

Retailer   country    
E-mail     Australia      10
           Austria        10
           Belgium        10
           Brazil         10
           Canada         10
           China          10
           Denmark        10
           Finland        10
           France         10
           Germany        10
           Italy          10
           Japan          10
           Korea          10
           Mexico         10
           Netherlands    10
           Singapore      10
           Spain          10
           Sweden         10
           Switzerland    10
           United         20
Fax        Australia      10
           Austria        10
           Belgium        10
           Brazil         10
           Canada         10
           China          10
           Denmark        10
           Finland        10
           France         10
           Germany        10
           Italy          10
           Japan          10
           Korea          10
           Mexico   

In [279]:
temp1["Order method type"] = temp1["Retailer"]
temp1 = temp1.drop(["Retailer"], axis=1)
temp1.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
735,2004,United,States,865351,930482,610494,7173.00,85.11,129.72,254856.69,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
736,2004,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
737,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
738,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
739,2004,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone


In [280]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["country", "Revenue", "Planned"])["Year"].count())

country      Revenue     Planned   
Australia    2485        2485.00       1
             3118.5      5670.00       1
             10977.75    14490.00      1
             19178.6     21385.00      1
             27004.34    28728.30      1
             28412.02    30224.76      1
             39013.17    41886.30      1
             56458.22    76794.24      1
             116330.76   131276.64     1
             141837.73   149307.72     1
             164434.36   171492.60     1
             411588.2    433264.80     1
             513252.95   540283.80     1
Austria      3549        5460.00       1
             15430.8     16660.00      1
             19467       24885.00      1
             28214.2     30275.00      1
             29506.89    32017.80      1
             38414.87    41228.40      1
             53926.08    57984.84      1
             57834.12    63816.30      1
             64323       67445.00      1
             118281.8    131276.64     1
             118736.6

In [281]:
temp2 = copy.deepcopy(temp1[temp1["Revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
735,2004,United,States,865351,930482,610494,7173.0,85.11,129.72,254856.69,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
750,2004,United,Kingdom,306426,329489,216179,2540.0,85.11,129.72,90246.20,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
777,2004,United,States,749657,806080,528874,6214.0,85.11,129.72,220783.42,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web
792,2004,United,Kingdom,476649,512524,336270,3951.0,85.11,129.72,140379.03,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web
798,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special


In [282]:
temp2.groupby(["country", "Revenue", "Planned"])["Year"].count()

country  Revenue  Planned   
United   Kingdom  0.00          1
                  18114.95      1
                  43366.75      1
                  55798.30      1
                  67328.10      1
                  69760.25      1
                  114823.85     1
                  153011.60     1
                  192116.28     1
                  225984.76     1
                  306425.60     1
                  393437.50     1
                  476648.64     1
                  647820.11     1
                  770310.73     1
                  804438.18     1
         States   840.00        1
                  6051.50       1
                  10877.28      1
                  13647.90      1
                  16699.20      1
                  17083.88      1
                  20407.86      1
                  27115.20      1
                  35895.20      1
                  37338.69      1
                  50483.16      1
                  65359.84      1
                  8

In [283]:
temp2["Retailer country"] = temp2["country"] + " " + temp2["Revenue"]
temp2 = temp2.drop(["country", "Revenue"], axis=1)

temp2["Revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Planned revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Product cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Unit price"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Quantity"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Gross profit"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Unit sale price"] = temp2["Unit.1"]
temp2 = temp2.drop(["Unit.1", "price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Quantity,Gross profit,Unit sale price
735,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United States,865351,930482,610494,85.11,129.72,7173.0,254856.69,120.64
750,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United Kingdom,306426,329489,216179,85.11,129.72,2540.0,90246.20,120.64
777,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United States,749657,806080,528874,85.11,129.72,6214.0,220783.42,120.64
792,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United Kingdom,476649,512524,336270,85.11,129.72,3951.0,140379.03,120.64
798,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [284]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
735,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United States,865351,930482,610494,7173.0,85.11,129.72,254856.69,120.64
750,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,United Kingdom,306426,329489,216179,2540.0,85.11,129.72,90246.20,120.64
777,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United States,749657,806080,528874,6214.0,85.11,129.72,220783.42,120.64
792,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Web,United Kingdom,476649,512524,336270,3951.0,85.11,129.72,140379.03,120.64
798,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
str11 = temp2

In [286]:
temp2 = copy.deepcopy(temp1[~temp1["Revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
736,2004,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
737,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
738,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
739,2004,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone
740,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone


In [287]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["country", "Revenue", "Planned"])["Year"].count())

country      Revenue     Planned   
Australia    2485.00     2485.00       1
             3118.50     5670.00       1
             10977.75    14490.00      1
             19178.60    21385.00      1
             27004.34    28728.30      1
             28412.02    30224.76      1
             39013.17    41886.30      1
             56458.22    76794.24      1
             116330.76   131276.64     1
             141837.73   149307.72     1
             164434.36   171492.60     1
             411588.20   433264.80     1
             513252.95   540283.80     1
Austria      3549.00     5460.00       1
             15430.80    16660.00      1
             19467.00    24885.00      1
             28214.20    30275.00      1
             29506.89    32017.80      1
             38414.87    41228.40      1
             53926.08    57984.84      1
             57834.12    63816.30      1
             64323.00    67445.00      1
             118281.80   131276.64     1
             118736.6

In [288]:
temp2["Retailer country"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Revenue"] = temp2["Revenue"]

temp2["Planned revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Product cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Gross profit"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit sale price"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1","Unit.1", "price", "Gross", "profit", "Unit.2", "sale", "price.1"], axis=1)

temp2.head()

,Year,Revenue,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Product cost,Unit cost,Unit price,Quantity,Gross profit,Unit sale price
736,2004,67799.7,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Canada,72902.6,47831.8,85.11,129.72,562,19967.86,120.64
737,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN
738,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,2004,419827,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Japan,451426,296183,85.11,129.72,3480,123644.40,120.64
740,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
736,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Canada,67799.7,72902.6,47831.8,562,85.11,129.72,19967.86,120.64
737,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
738,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Japan,419827,451426,296183,3480,85.11,129.72,123644.40,120.64
740,2004,Camping Equipment,Cooking Gear,TrailChef Deluxe Cook Set,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
str12 = temp2

In [300]:
temp = copy.deepcopy(tmp3[~tmp3.index.isin(tmp4.index.values)])
temp.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
0,2004,TrailChef,Water,Bag,Telephone,United,States,315044,437477,158372,...,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
1,2004,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
2,2004,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
3,2004,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
4,2004,TrailChef,Water,Bag,Telephone,Japan,181120,235237,89413.1,35696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [301]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["Product.2", "Order", "method", "type.1", ])["Year"].count())

Product.2   Order         method    type.1   
BugShield   Lotion        Lite      E-mail       63
                                    Fax          63
                                    Mail         63
                                    Sales        63
                                    Special      63
                                    Telephone    63
                                    Web          63
Compact     Relief        Kit       E-mail       63
                                    Fax          63
                                    Mail         63
                                    Sales        63
                                    Special      63
                                    Telephone    63
                                    Web          63
Course      Pro           Gloves    E-mail       63
                                    Fax          63
                                    Mail         63
                                    Sales        63
                  

In [302]:
temp["Product"] = temp["Product.2"] + " " + temp["Order"] + " " + temp["method"]
temp = temp.drop(["Product.2", "Order", "method"], axis=1)
temp.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
0,2004,Telephone,United,States,315044,437477,158372,66385,2.552857,6.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
1,2004,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.590000,7145.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
2,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
3,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
4,2004,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.590000,91707.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag


In [306]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["type.1", "Retailer"])["Year"].count())

type.1     Retailer   
E-mail     Australia       47
           Austria         47
           Belgium         47
           Brazil          47
           Canada          47
           China           47
           Denmark         47
           Finland         47
           France          47
           Germany         47
           Italy           47
           Japan           47
           Korea           47
           Mexico          47
           Netherlands     47
           Singapore       47
           Spain           47
           Sweden          47
           Switzerland     47
           United          94
Fax        Australia       47
           Austria         47
           Belgium         47
           Brazil          47
           Canada          47
           China           47
           Denmark         47
           Finland         47
           France          47
           Germany         47
           Italy           47
           Japan           47
           Korea 

In [307]:
temp1 = copy.deepcopy(temp[temp["Retailer"] == "visit"])
temp1.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
21,2004,Sales,visit,United,States,181288,222735,88348.1,33799.00,2.72,...,5.652222,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
22,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
23,2004,Sales,visit,Mexico,133617,169943,65215.1,25788,2.72,6.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
24,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
25,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag


In [308]:
temp1.groupby(["type.1", "Retailer", "country"])["Year"].count()

type.1  Retailer  country    
Sales   visit     Australia      47
                  Austria        47
                  Belgium        47
                  Brazil         47
                  Canada         47
                  China          47
                  Denmark        47
                  Finland        47
                  France         47
                  Germany        47
                  Italy          47
                  Japan          47
                  Korea          47
                  Mexico         47
                  Netherlands    47
                  Singapore      47
                  Spain          47
                  Sweden         47
                  Switzerland    47
                  United         94
Name: Year, dtype: int64

In [309]:
temp1["Order method type"] = temp1["type.1"] + " " + temp1["Retailer"]
temp1 = temp1.drop(["type.1", "Retailer"], axis=1)
temp1.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
21,2004,United,States,181288,222735,88348.1,33799.00,2.72,6.59,92939.600000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
22,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
23,2004,Mexico,133617,169943,65215.1,25788,2.72,6.59,68402.21,5.723333,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
24,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
25,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit


In [311]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["country", "Revenue", "Planned"])["Year"].count())

country      Revenue    Planned  
Australia    0          1191.33      1
             1886       1886.00      1
             1956       1956.00      1
             2790       2790.00      1
             4104       4104.00      1
             7521       7521.00      1
             7602.86    7758.06      1
             8057       8057.00      1
             8785       8785.00      1
             12714.48   12870.00     1
             18805.5    19181.61     1
             24850.6    32215.68     1
             27674.11   31552.11     1
             29500.6    30884.91     1
             30713.22   31882.42     1
             32669.14   34755.33     1
             33784.24   35941.12     1
             35259.84   35985.60     1
             36652      38520.00     1
             37295.9    62222.78     1
             45371.54   52360.65     1
             64072.99   65640.40     1
             64221.22   67598.56     1
             82051.2    84592.20     1
             82205.28   86533.

In [312]:
temp2 = copy.deepcopy(temp1[temp1["Revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
21,2004,United,States,181288,222735,88348.1,33799.0,2.72,6.59,92939.60,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
36,2004,United,Kingdom,11637.3,12250.8,5391.1,1859.0,2.90,6.59,6246.24,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
315,2004,United,States,96725.2,102887,68865.4,4323.0,15.93,23.80,27859.85,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit
330,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit
609,2004,United,States,302116,322549,205344,5872.0,34.97,54.93,96772.39,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit


In [313]:
temp2.groupby(["country", "Revenue", "Planned"])["Year"].count()

country  Revenue  Planned  
United   Kingdom  1817.00      1
                  2910.60      1
                  5711.36      1
                  6090.03      1
                  6807.08      1
                              ..
         States   186802.29    1
                  192573.48    1
                  204329.24    1
                  302116.23    1
                  327489.51    1
Name: Year, Length: 64, dtype: int64

In [314]:
temp2.columns

Index(['Year', 'country', 'Revenue', 'Planned', 'revenue', 'Product.3', 'cost',
       'Quantity', 'Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit',
       'Unit.2', 'sale', 'price.1', 'Product line', 'Product type', 'Product',
       'Order method type'],
      dtype='object')

In [316]:
temp2["Retailer country"] = temp2["country"] + " " + temp2["Revenue"]
temp2 = temp2.drop(["country", "Revenue"], axis=1)

temp2["Revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Planned revenue"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Product cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit price"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Gross profit"] = temp2["cost.1"]
temp2 = temp2.drop(["cost.1"], axis=1)

temp2["Unit sale price"] = temp2["Unit.1"]
temp2 = temp2.drop(['Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Quantity,Unit price,Gross profit,Unit sale price
21,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United States,181288,222735,88348.1,2.72,33799.0,6.59,92939.60,5.652222
36,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United Kingdom,11637.3,12250.8,5391.1,2.90,1859.0,6.59,6246.24,6.260000
315,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United States,96725.2,102887,68865.4,15.93,4323.0,23.80,27859.85,22.612000
330,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,2004,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit,United States,302116,322549,205344,34.97,5872.0,54.93,96772.39,42.792727


In [317]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
21,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United States,181288,222735,88348.1,33799.0,2.72,6.59,92939.60,5.652222
36,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,United Kingdom,11637.3,12250.8,5391.1,1859.0,2.90,6.59,6246.24,6.260000
315,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United States,96725.2,102887,68865.4,4323.0,15.93,23.80,27859.85,22.612000
330,2004,Camping Equipment,Cooking Gear,TrailChef Kitchen Kit,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,2004,Camping Equipment,Cooking Gear,TrailChef Cook Set,Sales visit,United States,302116,322549,205344,5872.0,34.97,54.93,96772.39,42.792727


In [318]:
str13 = temp2

In [319]:
temp2 = copy.deepcopy(temp1[~temp1["Revenue"].isin(["Kingdom", "States"])])
temp2.head()

,Year,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,cost.1,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
22,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
23,2004,Mexico,133617,169943,65215.1,25788,2.7200,6.59,68402.21,5.723333,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
24,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
25,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit
26,2004,Korea,186697,238492,91891.4,36190,2.6975,6.59,94805.19,5.585000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit


In [320]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["country", "Revenue", "Planned"])["Year"].count())

country      Revenue    Planned  
Australia    0.00       1191.33      1
             1886.00    1886.00      1
             1956.00    1956.00      1
             2790.00    2790.00      1
             4104.00    4104.00      1
             7521.00    7521.00      1
             7602.86    7758.06      1
             8057.00    8057.00      1
             8785.00    8785.00      1
             12714.48   12870.00     1
             18805.50   19181.61     1
             24850.60   32215.68     1
             27674.11   31552.11     1
             29500.60   30884.91     1
             30713.22   31882.42     1
             32669.14   34755.33     1
             33784.24   35941.12     1
             35259.84   35985.60     1
             36652.00   38520.00     1
             37295.90   62222.78     1
             45371.54   52360.65     1
             64072.99   65640.40     1
             64221.22   67598.56     1
             82051.20   84592.20     1
             82205.28   86533.

In [322]:
temp2["Retailer country"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Planned revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Product cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Gross profit"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit sale price"] = temp2["cost.1"]
temp2 = temp2.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Revenue,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Product cost,Unit cost,Unit price,Quantity,Gross profit,Unit sale price
22,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2004,133617,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Mexico,169943,65215.1,2.7200,6.59,25788,68402.21,5.723333
24,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2004,186697,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Korea,238492,91891.4,2.6975,6.59,36190,94805.19,5.585000


In [323]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
22,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Mexico,133617,169943,65215.1,25788,2.7200,6.59,68402.21,5.723333
24,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Sales visit,Korea,186697,238492,91891.4,36190,2.6975,6.59,94805.19,5.585000


In [324]:
str14 = temp2

In [325]:
temp1 = copy.deepcopy(temp[temp["Retailer"] != "visit"])
temp1.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
0,2004,Telephone,United,States,315044,437477,158372,66385,2.552857,6.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
1,2004,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.590000,7145.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
2,2004,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
3,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag
4,2004,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.590000,91707.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag


In [327]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["type.1", "Retailer"])["Year"].count())

type.1     Retailer   
E-mail     Australia      47
           Austria        47
           Belgium        47
           Brazil         47
           Canada         47
           China          47
           Denmark        47
           Finland        47
           France         47
           Germany        47
           Italy          47
           Japan          47
           Korea          47
           Mexico         47
           Netherlands    47
           Singapore      47
           Spain          47
           Sweden         47
           Switzerland    47
           United         94
Fax        Australia      47
           Austria        47
           Belgium        47
           Brazil         47
           Canada         47
           China          47
           Denmark        47
           Finland        47
           France         47
           Germany        47
           Italy          47
           Japan          47
           Korea          47
           Mexico   

In [328]:
temp1["Order method type"] = temp1["type.1"]
temp1 = temp1.drop(["type.1"], axis=1)
temp1.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
0,2004,United,States,315044,437477,158372,66385,2.552857,6.59,156672.570,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
1,2004,Canada,13444.7,14313.5,6298.8,2172,2.9,6.590000,7145.88,6.190,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
2,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
3,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
4,2004,Japan,181120,235237,89413.1,35696,2.657,6.590000,91707.18,5.488,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone


In [329]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["Retailer", "country", "Revenue"])["Year"].count())

6572.42      1
             76153.92   83304.00      1
             85140      95285.76      1
             87558.04   92172.54      1
             93235.4    131583.06     1
             97534.43   104853.49     1
             100345.12  145613.16     1
             106352.82  117570.60     1
             112448.96  124196.73     1
             117431.1   126262.07     1
             125140.74  132122.34     1
             125414.17  133420.96     1
             129256.56  132912.00     1
             135474.43  142408.77     1
             143364.09  154914.20     1
             148678.44  158178.13     1
             176506.43  182450.80     1
             182692.85  233846.15     1
             209527.16  279442.36     1
             216808.9   248162.60     1
             228141.83  245736.96     1
             235074.65  242831.40     1
             283422.51  340755.72     1
             299461.86  315226.29     1
             315521.15  344466.03     1
             335861.85  3

In [334]:
temp2 = copy.deepcopy(temp1[temp1["country"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
0,2004,United,States,315044,437477,158372,66385,2.552857,6.59,156672.57,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
15,2004,United,Kingdom,160227,220996,80307.1,33535,2.576000,6.59,79920.02,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
42,2004,United,States,340867,407124,164590,61779,2.778500,6.59,176276.82,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web
57,2004,United,Kingdom,168290,199446,80612.1,30265,2.775385,6.59,87678.26,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web
63,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special


In [335]:
temp2.groupby(["Retailer", "country", "Revenue"])["Year"].count()

Retailer  country  Revenue   
United    Kingdom  366.00        1
                   1817.00       1
                   2328.48       1
                   2840.04       1
                   2857.68       1
                                ..
          States   1222803.54    1
                   1277590.76    1
                   1299690.94    1
                   1657838.21    1
                   1681973.31    1
Name: Year, Length: 222, dtype: int64

In [337]:
temp2["Retailer country"] = temp2["Retailer"] + " " + temp2["country"]
temp2 = temp2.drop(["Retailer", "country"], axis=1)

temp2["Planned revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Product cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Gross profit"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit sale price"] = temp2["cost.1"]
temp2 = temp2.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Revenue,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Product cost,Unit price,Quantity,Unit cost,Gross profit,Unit sale price
0,2004,315044,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United States,437477,158372,6.59,66385,2.552857,156672.57,5.195714
15,2004,160227,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United Kingdom,220996,80307.1,6.59,33535,2.576000,79920.02,5.270000
42,2004,340867,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United States,407124,164590,6.59,61779,2.778500,176276.82,5.838000
57,2004,168290,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United Kingdom,199446,80612.1,6.59,30265,2.775385,87678.26,5.877692
63,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
0,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United States,315044,437477,158372,66385,2.552857,6.59,156672.57,5.195714
15,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,United Kingdom,160227,220996,80307.1,33535,2.576000,6.59,79920.02,5.270000
42,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United States,340867,407124,164590,61779,2.778500,6.59,176276.82,5.838000
57,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Web,United Kingdom,168290,199446,80612.1,30265,2.775385,6.59,87678.26,5.877692
63,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
str15 = temp2

In [345]:
temp2 = copy.deepcopy(temp1[~temp1["country"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
1,2004,Canada,13444.7,14313.5,6298.8,2172,2.9,6.59,7145.88,6.190,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
2,2004,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
3,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
4,2004,Japan,181120,235237,89413.1,35696,2.657,6.59,91707.18,5.488,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone
5,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone


In [346]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["Retailer", "country"])["Year"].count())

00       1
             1050.00      1
             1794.00      1
             1818.00      1
             2250.00      1
             3338.94      1
             3546.00      1
             3657.00      1
             4270.29      1
             4781.07      1
             4812.00      1
             5365.50      1
             6018.18      1
             6156.36      1
             6348.78      1
             6603.76      1
             6797.00      1
             7411.88      1
             7452.00      1
             7956.00      1
             8130.00      1
             8376.30      1
             9139.34      1
             9173.34      1
             9913.80      1
             10529.04     1
             12399.33     1
             13234.22     1
             13408.31     1
             13636.70     1
             13884.17     1
             13997.34     1
             14186.82     1
             14761.73     1
             16406.30     1
             16520.00     1
         

In [347]:
temp2.columns

Index(['Year', 'Retailer', 'country', 'Revenue', 'Planned', 'revenue',
       'Product.3', 'cost', 'Quantity', 'Unit', 'cost.1', 'Unit.1', 'price',
       'Gross', 'profit', 'Unit.2', 'sale', 'price.1', 'Product line',
       'Product type', 'Product', 'Order method type'],
      dtype='object')

In [348]:
temp2["Retailer country"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Planned revenue"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Product cost"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Gross profit"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit price"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit sale price"] = temp2["Unit"]
temp2 = temp2.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Revenue,Product cost,Gross profit,Quantity,Unit cost,Unit price,Unit sale price
1,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Canada,14313.5,13444.7,6298.8,7145.88,2172,2.9,6.59,6.190
2,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Japan,235237,181120,89413.1,91707.18,35696,2.657,6.59,5.488
5,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
1,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Canada,13444.7,14313.5,6298.8,2172,2.9,6.59,7145.88,6.190
2,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Japan,181120,235237,89413.1,35696,2.657,6.59,91707.18,5.488
5,2004,Camping Equipment,Cooking Gear,TrailChef Water Bag,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [350]:
str16 = temp2

In [353]:
temp = copy.deepcopy(tmp2[~tmp2.index.isin(tmp3.index.values)])
temp.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
147,2004,TrailChef,Canteen,Telephone,United,States,209592,235238,124284,18774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
148,2004,TrailChef,Canteen,Telephone,Canada,8489.36,12129,6408.16,968,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
149,2004,TrailChef,Canteen,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
150,2004,TrailChef,Canteen,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear
151,2004,TrailChef,Canteen,Telephone,Japan,107959,124486,65769.7,9935,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear


In [357]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["Product.2", "Order", "method"])["Year"].count())

Product.2   Order      method   
Aloe        Relief     E-mail       63
                       Fax          63
                       Mail         63
                       Sales        63
                       Special      63
                       Telephone    63
                       Web          63
BugShield   Extreme    E-mail       63
                       Fax          63
                       Mail         63
                       Sales        63
                       Special      63
                       Telephone    63
                       Web          63
            Lotion     E-mail       63
                       Fax          63
                       Mail         63
                       Sales        63
                       Special      63
                       Telephone    63
                       Web          63
            Natural    E-mail       63
                       Fax          63
                       Mail         63
                       Sales   

In [358]:
temp["Product"] = temp["Product.2"] + " " + ["Order"]
temp = temp.drop(["Product.2", "Order"], axis=1)
temp.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
147,2004,Telephone,United,States,209592,235238,124284,18774,6.62,12.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
148,2004,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
149,2004,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
150,2004,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
151,2004,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order


In [360]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["method", "type.1"])["Year"].count())

method     type.1     
E-mail     Australia        58
           Austria          58
           Belgium          58
           Brazil           58
           Canada           58
           China            58
           Denmark          58
           Finland          58
           France           58
           Germany          58
           Italy            58
           Japan            58
           Korea            58
           Mexico           58
           Netherlands      58
           Singapore        58
           Spain            58
           Sweden           58
           Switzerland      58
           United          116
Fax        Australia        58
           Austria          58
           Belgium          58
           Brazil           58
           Canada           58
           China            58
           Denmark          58
           Finland          58
           France           58
           Germany          58
           Italy            58
           Japan

In [365]:
temp1 = copy.deepcopy(temp[temp["type.1"] == "visit"])
temp1.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
168,2004,Sales,visit,United,States,58425.6,66045.6,34894,5271,6.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
169,2004,Sales,visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
170,2004,Sales,visit,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
171,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
172,2004,Sales,visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order


In [366]:
temp1.groupby(["method", "type.1", "Retailer"])["Year"].count()

method  type.1  Retailer   
Sales   visit   Australia       58
                Austria         58
                Belgium         58
                Brazil          58
                Canada          58
                China           58
                Denmark         58
                Finland         58
                France          58
                Germany         58
                Italy           58
                Japan           58
                Korea           58
                Mexico          58
                Netherlands     58
                Singapore       58
                Spain           58
                Sweden          58
                Switzerland     58
                United         116
Name: Year, dtype: int64

In [367]:
temp1["Order method type"] = temp1["method"] + " " + temp1["type.1"]
temp1 = temp1.drop(["method", "type.1"], axis=1)
temp1.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
168,2004,United,States,58425.6,66045.6,34894,5271,6.62,12.53,23531.5700,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
169,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
170,2004,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.6375,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
171,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
172,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit


In [368]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["Retailer", "country", "Revenue"])["Year"].count())

8    1
             478081.62  502705.44    1
Belgium      0          203.00       1
             186        186.00       1
             372        372.00       1
             444        444.00       1
             831.57     831.57       1
             1610.68    1610.68      1
             2214       2214.00      1
             2298.48    2298.48      1
             2566.27    2566.27      1
             4493.2     5842.15      1
             4860       5012.00      1
             5851.7     5965.80      1
             7084       7084.00      1
             7095.62    7175.00      1
             10207.74   10207.74     1
             10350.12   12276.30     1
             10754.12   12619.75     1
             13612.98   14034.00     1
             14282.07   17729.95     1
             15063.5    15533.50     1
             15205.68   15516.00     1
             15248.65   15721.35     1
             16152.96   16872.00     1
             18542.73   18827.04     1
             23039

In [369]:
temp2 = copy.deepcopy(temp1[temp1["country"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
168,2004,United,States,58425.6,66045.6,34894,5271,6.620,12.53,23531.57,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
183,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
462,2004,United,States,11644.4,12118.3,2814.35,3311,0.850,3.66,8830.01,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
477,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
1197,2004,United,States,176285,216266,87415.3,16359,5.425,13.22,88869.88,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit


In [370]:
temp2.groupby(["Retailer", "country", "Revenue"])["Year"].count()

Retailer  country  Revenue  
United    Kingdom  264.00       1
                   2085.47      1
                   2639.00      1
                   5101.39      1
                   7668.00      1
                               ..
          States   294067.80    1
                   368122.59    1
                   387494.73    1
                   448115.54    1
                   957223.56    1
Name: Year, Length: 82, dtype: int64

In [371]:
temp2.columns

Index(['Year', 'Retailer', 'country', 'Revenue', 'Planned', 'revenue',
       'Product.3', 'cost', 'Quantity', 'Unit', 'cost.1', 'Unit.1', 'price',
       'Gross', 'profit', 'Unit.2', 'sale', 'price.1', 'Product line',
       'Product type', 'Product', 'Order method type'],
      dtype='object')

In [372]:
temp2["Retailer country"] = temp2["Retailer"] + " " + temp2["country"]
temp2 = temp2.drop(["Retailer", "country"], axis=1)

temp2["Planned revenue"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Product cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit cost"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Gross profit"] = temp2["Unit"]
temp2 = temp2.drop(["Unit"], axis=1)

temp2["Unit sale price"] = temp2["cost.1"]
temp2 = temp2.drop(['cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Revenue,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Product cost,Unit price,Quantity,Unit cost,Gross profit,Unit sale price
168,2004,58425.6,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,66045.6,34894,12.53,5271,6.620,23531.57,11.486667
183,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,2004,11644.4,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,12118.3,2814.35,3.66,3311,0.850,8830.01,3.526667
477,2004,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2004,176285,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,216266,87415.3,13.22,16359,5.425,88869.88,11.303750


In [373]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
168,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,58425.6,66045.6,34894,5271,6.620,12.53,23531.57,11.486667
183,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,11644.4,12118.3,2814.35,3311,0.850,3.66,8830.01,3.526667
477,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,United States,176285,216266,87415.3,16359,5.425,13.22,88869.88,11.303750


In [374]:
str17 = temp2

In [375]:
temp2 = copy.deepcopy(temp1[~temp1["country"].isin(["Kingdom", "States"])])
temp2.head()

,Year,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,Unit,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
169,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
170,2004,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.637500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
171,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
172,2004,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit
173,2004,Korea,79294.1,88273.9,46637.9,7045,6.62,12.53,32656.25,11.545714,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit


In [377]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp2.groupby(["Retailer", "country"])["Year"].count())

Retailer     country  
Australia    1326.00      1
             1746.82      1
             2058.00      1
             2212.29      1
             2862.00      1
             5194.73      1
             5897.24      1
             7628.88      1
             9525.85      1
             10906.00     1
             10978.32     1
             13726.20     1
             14302.56     1
             16087.68     1
             16576.00     1
             16825.32     1
             20562.29     1
             21543.21     1
             24486.03     1
             25928.10     1
             27518.40     1
             28418.04     1
             35306.28     1
             38113.24     1
             43586.48     1
             51521.12     1
             51867.54     1
             52047.80     1
             57778.56     1
             58150.40     1
             95195.01     1
             95519.04     1
             270663.78    1
Austria      0.00         1
             266.73      

In [378]:
temp2["Retailer country"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Planned revenue"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Product cost"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Gross profit"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit price"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit sale price"] = temp2["Unit"]
temp2 = temp2.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Revenue,Product cost,Gross profit,Quantity,Unit cost,Unit price,Unit sale price
169,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Mexico,58853.4,54227.7,31094.1,23133.53,4697,6.62,12.53,11.637500
171,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Korea,88273.9,79294.1,46637.9,32656.25,7045,6.62,12.53,11.545714


In [379]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
169,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Mexico,54227.7,58853.4,31094.1,4697,6.62,12.53,23133.53,11.637500
171,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2004,Camping Equipment,Cooking Gear,TrailChef Order,Sales visit,Korea,79294.1,88273.9,46637.9,7045,6.62,12.53,32656.25,11.545714


In [380]:
str18 = temp2

In [381]:
temp1 = copy.deepcopy(temp[temp["type.1"] != "visit"])
temp1.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
147,2004,Telephone,United,States,209592,235238,124284,18774,6.62,12.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
148,2004,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
149,2004,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
150,2004,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order
151,2004,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order


In [383]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["method", "type.1"])["Year"].count())

method     type.1     
E-mail     Australia       58
           Austria         58
           Belgium         58
           Brazil          58
           Canada          58
           China           58
           Denmark         58
           Finland         58
           France          58
           Germany         58
           Italy           58
           Japan           58
           Korea           58
           Mexico          58
           Netherlands     58
           Singapore       58
           Spain           58
           Sweden          58
           Switzerland     58
           United         116
Fax        Australia       58
           Austria         58
           Belgium         58
           Brazil          58
           Canada          58
           China           58
           Denmark         58
           Finland         58
           France          58
           Germany         58
           Italy           58
           Japan           58
           Korea 

In [384]:
temp1["Order method type"] = temp1["method"]
temp1 = temp1.drop(["method"], axis=1)
temp1.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
147,2004,United,States,209592,235238,124284,18774,6.62,12.53,85308.180000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
148,2004,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
149,2004,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
150,2004,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
151,2004,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone


In [385]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["type.1", "Retailer", "country"])["Year"].count())

      1
             74120.94    78017.94      1
             75177.26    87844.65      1
             78532.98    81012.00      1
             82766.66    91647.95      1
             88353.44    91266.00      1
             90014.84    95765.67      1
             93853.15    103986.47     1
             97067.55    102481.44     1
             97804.12    101870.49     1
             101970.43   134759.94     1
             105382.39   120039.45     1
             112893.36   118788.00     1
             114870.52   121676.00     1
             126971.54   153497.42     1
             136123.98   140334.00     1
             136898.11   146112.33     1
             148077.68   152081.30     1
             157381.91   203892.06     1
             162986.6    173390.00     1
             171975.2    209180.76     1
             173961.7    221514.32     1
             175050.8    184264.00     1
             180597.96   190105.38     1
             181875.74   188579.04     1
        

In [386]:
temp2 = copy.deepcopy(temp1[temp1["Retailer"].isin(["Kingdom", "States"])])
temp2.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
147,2004,United,States,209592,235238,124284,18774,6.62,12.53,85308.18,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
162,2004,United,Kingdom,108404,123934,65478.4,9891,6.62,12.53,42925.27,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
189,2004,United,States,156644,176335,93163.3,14073,6.62,12.53,63481.21,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Web
204,2004,United,Kingdom,52519.3,57487.6,30372.6,4588,6.62,12.53,22146.74,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Web
210,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Special


In [387]:
temp2.groupby(["type.1", "Retailer", "country"])["Year"].count()

type.1  Retailer  country   
United  Kingdom   276.00        1
                  1265.66       1
                  3963.36       1
                  4788.00       1
                  5416.07       1
                               ..
        States    1053294.52    1
                  1414240.68    1
                  1620991.92    1
                  2199235.05    1
                  3477909.78    1
Name: Year, Length: 292, dtype: int64

In [388]:
temp2.columns

Index(['Year', 'type.1', 'Retailer', 'country', 'Revenue', 'Planned',
       'revenue', 'Product.3', 'cost', 'Quantity', 'Unit', 'cost.1', 'Unit.1',
       'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1', 'Product line',
       'Product type', 'Product', 'Order method type'],
      dtype='object')

In [389]:
temp2["Retailer country"] = temp2["type.1"] + " " + temp2["Retailer"]
temp2 = temp2.drop(["type.1", "Retailer"], axis=1)

temp2["Planned revenue"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Product cost"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Gross profit"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit price"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit sale price"] = temp2["Unit"]
temp2 = temp2.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Revenue,Product cost,Gross profit,Quantity,Unit cost,Unit price,Unit sale price
147,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,United States,235238,209592,124284,85308.18,18774,6.62,12.53,11.427000
162,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,United Kingdom,123934,108404,65478.4,42925.27,9891,6.62,12.53,11.316667
189,2004,Camping Equipment,Cooking Gear,TrailChef Order,Web,United States,176335,156644,93163.3,63481.21,14073,6.62,12.53,11.500000
204,2004,Camping Equipment,Cooking Gear,TrailChef Order,Web,United Kingdom,57487.6,52519.3,30372.6,22146.74,4588,6.62,12.53,11.652500
210,2004,Camping Equipment,Cooking Gear,TrailChef Order,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [390]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
147,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,United States,209592,235238,124284,18774,6.62,12.53,85308.18,11.427000
162,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,United Kingdom,108404,123934,65478.4,9891,6.62,12.53,42925.27,11.316667
189,2004,Camping Equipment,Cooking Gear,TrailChef Order,Web,United States,156644,176335,93163.3,14073,6.62,12.53,63481.21,11.500000
204,2004,Camping Equipment,Cooking Gear,TrailChef Order,Web,United Kingdom,52519.3,57487.6,30372.6,4588,6.62,12.53,22146.74,11.652500
210,2004,Camping Equipment,Cooking Gear,TrailChef Order,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [391]:
str19 = temp2

In [398]:
temp2 = copy.deepcopy(temp1[~temp1["Retailer"].isin(["Kingdom", "States"])])
temp2.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
148,2004,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
149,2004,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
150,2004,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
151,2004,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone
152,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Cooking Gear,TrailChef Order,Telephone


In [399]:
temp2["Retailer country"] = temp2["type.1"]
temp2 = temp2.drop(["type.1"], axis=1)

temp2["Product cost"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Planned revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Unit sale price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Unit cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit price"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Gross profit"] = temp2["cost"]
temp2 = temp2.drop(['cost', 'Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Product cost,Revenue,Planned revenue,Unit sale price,Quantity,Unit cost,Unit price,Gross profit
148,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Canada,6408.16,8489.36,12129,8.770000,968,6.62,12.53,2081.20
149,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Mexico,9029.68,11962.3,17090.9,8.770000,1364,6.62,12.53,2932.60
150,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Brazil,5977.86,7919.31,11314.6,8.770000,903,6.62,12.53,1941.45
151,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Japan,65769.7,107959,124486,11.286923,9935,6.62,12.53,42189.47
152,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [400]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
148,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Canada,8489.36,12129,6408.16,968,6.62,12.53,2081.20,8.770000
149,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Mexico,11962.3,17090.9,9029.68,1364,6.62,12.53,2932.60,8.770000
150,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Brazil,7919.31,11314.6,5977.86,903,6.62,12.53,1941.45,8.770000
151,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Japan,107959,124486,65769.7,9935,6.62,12.53,42189.47,11.286923
152,2004,Camping Equipment,Cooking Gear,TrailChef Order,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [401]:
str20 = temp2

In [403]:
temp = copy.deepcopy(tmp1[~tmp1.index.isin(tmp2.index.values)])
temp.head()

,Year,Product.2,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,...,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type
2499,2004,Hibernator,Telephone,United,States,968871,1130591,662200,7700,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2500,2004,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2501,2004,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2502,2004,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags
2503,2004,Hibernator,Telephone,Japan,624589,720054,421744,4904,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags


In [404]:
temp.groupby(["Product.2", "Order"])["Year"].count()

Product.2   Order    
Hibernator  E-mail       63
            Fax          63
            Mail         63
            Sales        63
            Special      63
            Telephone    63
            Web          63
Name: Year, dtype: int64

In [405]:
temp["Product"] = temp["Product.2"]
temp = temp.drop(["Product.2"], axis=1)
temp.head()

,Year,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2499,2004,Telephone,United,States,968871,1130591,662200,7700,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2500,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2501,2004,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2502,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2503,2004,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator


In [407]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp.groupby(["Order", "method"])["Year"].count())

Order      method     
E-mail     Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico          3
           Netherlands     3
           Singapore       3
           Spain           3
           Sweden          3
           Switzerland     3
           United          6
Fax        Australia       3
           Austria         3
           Belgium         3
           Brazil          3
           Canada          3
           China           3
           Denmark         3
           Finland         3
           France          3
           Germany         3
           Italy           3
           Japan           3
           Korea           3
           Mexico   

In [410]:
temp1 = copy.deepcopy(temp[temp["method"] == "visit"])
temp1.head()

,Year,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2520,2004,Sales,visit,United,States,765609,850733,498284,5794,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2521,2004,Sales,visit,Canada,37551.2,40378.2,23650,275,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2522,2004,Sales,visit,Mexico,385230,439315,257312,2992,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2523,2004,Sales,visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2524,2004,Sales,visit,Japan,46700.1,50215.9,29412,342,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator


In [411]:
temp1["Order method type"] = temp1["Order"] + " " + temp1["method"]
temp1 = temp1.drop(["Order", "method"], axis=1)
temp1.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2520,2004,United,States,765609,850733,498284,5794,86,146.83,267325.1900,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2521,2004,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2522,2004,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2523,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2524,2004,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit


In [412]:
temp1.groupby(["type.1", "Retailer", "country"])["Year"].count()

type.1       Retailer   country  
Australia    67234.18   70772.06     1
             86374.11   91034.60     1
Austria      17032.28   17032.28     1
             212119.98  222006.96    1
Belgium      43149.8    46398.28     1
             45194.76   47572.92     1
             55760.08   59319.32     1
Canada       37551.25   40378.25     1
             38231.54   40671.91     1
China        83502.1    88832.15     1
             164040.24  172672.08    1
Finland      86685.48   108360.54    1
France       60120.19   63283.73     1
             148976.05  160191.53    1
             158584.98  168707.67    1
Germany      32158.66   34211.39     1
             339277.39  385428.75    1
Italy        48859.08   51977.82     1
             296947.8   344756.84    1
Japan        46700.1    50215.86     1
             53564.16   56382.72     1
Korea        63765.24   67835.46     1
             590417.14  673949.70    1
Mexico       92760.85   97641.95     1
             385229.62  439315

In [413]:
temp2 = copy.deepcopy(temp1[temp1["Retailer"].isin(["Kingdom", "States"])])
temp2.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2520,2004,United,States,765609,850733,498284,5794,86,146.83,267325.19,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2535,2004,United,Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
23688,2005,United,States,234082,249024,145856,1696,86,146.83,88225.92,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
23703,2005,United,Kingdom,60452.8,64311.5,37668,438,86,146.83,22784.76,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
44856,2006,United,States,589829,616392,361028,4198,86,146.83,228800.88,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit


In [414]:
temp2["Retailer country"] = temp2["type.1"] + " " + temp2["Retailer"]
temp2 = temp2.drop(["type.1", "Retailer"], axis=1)

temp2["Planned revenue"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Product cost"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Gross profit"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit cost"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit price"] = temp2["cost"]
temp2 = temp2.drop(["cost"], axis=1)

temp2["Unit sale price"] = temp2["Unit"]
temp2 = temp2.drop(['Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Planned revenue,Revenue,Product cost,Gross profit,Quantity,Unit cost,Unit price,Unit sale price
2520,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,850733,765609,498284,267325.19,5794,86,146.83,134.004667
2535,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23688,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,249024,234082,145856,88225.92,1696,86,146.83,138.020000
23703,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,64311.5,60452.8,37668,22784.76,438,86,146.83,138.020000
44856,2006,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,616392,589829,361028,228800.88,4198,86,146.83,140.305556


In [415]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2520,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,765609,850733,498284,5794,86,146.83,267325.19,134.004667
2535,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23688,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,234082,249024,145856,1696,86,146.83,88225.92,138.020000
23703,2005,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United Kingdom,60452.8,64311.5,37668,438,86,146.83,22784.76,138.020000
44856,2006,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,United States,589829,616392,361028,4198,86,146.83,228800.88,140.305556


In [416]:
str21 = temp2

In [417]:
temp2 = copy.deepcopy(temp1[~temp1["Retailer"].isin(["Kingdom", "States"])])
temp2.head()

,Year,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,Quantity,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2521,2004,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2522,2004,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2523,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2524,2004,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit
2525,2004,Korea,590417,673950,394740,4590,86,146.83,195677.14,131.7775,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Sales visit


In [418]:
temp2["Retailer country"] = temp2["type.1"]
temp2 = temp2.drop(["type.1"], axis=1)

temp2["Product cost"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Planned revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Unit sale price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Unit cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit price"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Gross profit"] = temp2["cost"]
temp2 = temp2.drop(['cost', 'Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Product cost,Revenue,Planned revenue,Unit sale price,Quantity,Unit cost,Unit price,Gross profit
2521,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Canada,23650,37551.2,40378.2,136.5500,275,86,146.83,13901.25
2522,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Mexico,257312,385230,439315,131.7775,2992,86,146.83,127917.62
2523,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Japan,29412,46700.1,50215.9,136.5500,342,86,146.83,17288.10
2525,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Korea,394740,590417,673950,131.7775,4590,86,146.83,195677.14


In [419]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2521,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Canada,37551.2,40378.2,23650,275,86,146.83,13901.25,136.5500
2522,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Mexico,385230,439315,257312,2992,86,146.83,127917.62,131.7775
2523,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2524,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Japan,46700.1,50215.9,29412,342,86,146.83,17288.10,136.5500
2525,2004,Camping Equipment,Sleeping Bags,Hibernator,Sales visit,Korea,590417,673950,394740,4590,86,146.83,195677.14,131.7775


In [420]:
str22 = temp2

In [421]:
temp1 = copy.deepcopy(temp[temp["method"] != "visit"])
temp1.head()

,Year,Order,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,...,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product
2499,2004,Telephone,United,States,968871,1130591,662200,7700,86,146.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2500,2004,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2501,2004,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2502,2004,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator
2503,2004,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator


In [423]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["Order", "method"])["Year"].count())

Order      method     
E-mail     Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands    3
           Singapore      3
           Spain          3
           Sweden         3
           Switzerland    3
           United         6
Fax        Australia      3
           Austria        3
           Belgium        3
           Brazil         3
           Canada         3
           China          3
           Denmark        3
           Finland        3
           France         3
           Germany        3
           Italy          3
           Japan          3
           Korea          3
           Mexico         3
           Netherlands   

In [424]:
temp1["Order method type"] = temp1["Order"]
temp1 = temp1.drop(["Order"], axis=1)
temp1.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2499,2004,United,States,968871,1130591,662200,7700,86,146.83,306670.750,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2500,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2501,2004,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2502,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2503,2004,Japan,624589,720054,421744,4904,86,146.83,202845,130.823,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone


In [426]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp1.groupby(["method", "type.1", "Retailer"])["Year"].count())

method       type.1      Retailer  
Australia    32572.72    34651.88      1
             146853.28   156227.12     1
             169729.38   179719.92     1
             184545.27   194256.09     1
             562833.34   585264.38     1
Austria      32536.42    40671.91      1
             118327.56   136698.73     1
             139309.19   147123.66     1
             164542.75   176930.15     1
             326555.32   347399.78     1
             426845.6    443720.26     1
Belgium      20790.42    25988.91      1
             129202.14   144040.23     1
             381275.89   403635.67     1
             525479.8    547969.56     1
Brazil       139144.45   149619.77     1
             282650.9    318914.76     1
             508744.7    539306.59     1
             673867.44   702434.72     1
Canada       42193.95    45370.47      1
             45408.58    48307.07      1
             91093.2     96907.80      1
             123577.75   132881.15     1
             129725.7

In [427]:
temp2 = copy.deepcopy(temp1[temp1["type.1"].isin(["Kingdom", "States"])])
temp2.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2499,2004,United,States,968871,1130591,662200,7700,86,146.83,306670.75,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2514,2004,United,Kingdom,558875,632397,370402,4307,86,146.83,188472.97,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2541,2004,United,States,833381,906528,530964,6174,86,146.83,302417.07,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Web
2556,2004,United,Kingdom,478974,525358,307708,3578,86,146.83,171265.63,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Web
2562,2004,United,States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Special


In [429]:
temp2["Retailer country"] = temp2["method"] + " " + temp2["type.1"]
temp2 = temp2.drop(["method", "type.1"], axis=1)

temp2["Product cost"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Planned revenue"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Unit sale price"] = temp2["Quantity"]
temp2 = temp2.drop(["Quantity"], axis=1)

temp2["Quantity"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Unit cost"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Unit price"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Gross profit"] = temp2["cost"]
temp2 = temp2.drop(['cost', 'Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Product cost,Revenue,Planned revenue,Unit sale price,Quantity,Unit cost,Unit price,Gross profit
2499,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United States,662200,968871,1130591,129.732143,7700,86,146.83,306670.75
2514,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United Kingdom,370402,558875,632397,131.777500,4307,86,146.83,188472.97
2541,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United States,530964,833381,906528,135.356875,6174,86,146.83,302417.07
2556,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United Kingdom,307708,478974,525358,134.959167,3578,86,146.83,171265.63
2562,2004,Camping Equipment,Sleeping Bags,Hibernator,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [430]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2499,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United States,968871,1130591,662200,7700,86,146.83,306670.75,129.732143
2514,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,United Kingdom,558875,632397,370402,4307,86,146.83,188472.97,131.777500
2541,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United States,833381,906528,530964,6174,86,146.83,302417.07,135.356875
2556,2004,Camping Equipment,Sleeping Bags,Hibernator,Web,United Kingdom,478974,525358,307708,3578,86,146.83,171265.63,134.959167
2562,2004,Camping Equipment,Sleeping Bags,Hibernator,Special,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [431]:
str23 = temp2

In [432]:
temp2 = copy.deepcopy(temp1[~temp1["type.1"].isin(["Kingdom", "States"])])
temp2.head()

,Year,method,type.1,Retailer,country,Revenue,Planned,revenue,Product.3,cost,...,price,Gross,profit,Unit.2,sale,price.1,Product line,Product type,Product,Order method type
2500,2004,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2501,2004,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2502,2004,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2503,2004,Japan,624589,720054,421744,4904,86,146.83,202845,130.823,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone
2504,2004,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone


In [434]:
temp2["Retailer country"] = temp2["method"]
temp2 = temp2.drop(["method"], axis=1)

temp2["Quantity"] = temp2["Revenue"]
temp2 = temp2.drop(["Revenue"], axis=1)

temp2["Revenue"] = temp2["type.1"]
temp2 = temp2.drop(["type.1"], axis=1)

temp2["Planned revenue"] = temp2["Retailer"]
temp2 = temp2.drop(["Retailer"], axis=1)

temp2["Product cost"] = temp2["country"]
temp2 = temp2.drop(["country"], axis=1)

temp2["Unit cost"] = temp2["Planned"]
temp2 = temp2.drop(["Planned"], axis=1)

temp2["Unit price"] = temp2["revenue"]
temp2 = temp2.drop(["revenue"], axis=1)

temp2["Gross profit"] = temp2["Product.3"]
temp2 = temp2.drop(["Product.3"], axis=1)

temp2["Unit sale price"] = temp2["cost"]
temp2 = temp2.drop(['cost', 'Unit', 'cost.1', 'Unit.1', 'price', 'Gross', 'profit', 'Unit.2', 'sale', 'price.1'], axis=1)

temp2.head()

,Year,Quantity,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Unit cost,Unit price,Gross profit,Unit sale price
2500,2004,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2501,2004,195,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,86,146.83,9857.25,136.550
2502,2004,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,2004,4904,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Japan,624589,720054,421744,86,146.83,202845,130.823
2504,2004,NaN,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
temp2 = temp2[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country', 'Revenue', 'Planned revenue', 'Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]

temp2.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
2500,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2501,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Mexico,26627.2,28631.8,16770,195,86,146.83,9857.25,136.550
2502,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Japan,624589,720054,421744,4904,86,146.83,202845,130.823
2504,2004,Camping Equipment,Sleeping Bags,Hibernator,Telephone,Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [436]:
str24 = temp2

In [437]:
tmp

,Year,Product.1,type,Product.2,Order,method,type.1,Retailer,country,Revenue,...,Unit,cost.1,Unit.1,price,Gross,profit,Unit.2,sale,price.1,Product line
0,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,United,States,315044,...,156672.570,5.195714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
1,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Canada,13444.7,14313.5,...,6.190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
2,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Mexico,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
3,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Brazil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
4,2004,Cooking,Gear,TrailChef,Water,Bag,Telephone,Japan,181120,235237,...,5.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,2007,Tents,Star,Gazer,3,E-mail,Sweden,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
65531,2007,Tents,Star,Gazer,3,E-mail,Finland,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
65532,2007,Tents,Star,Gazer,3,E-mail,Denmark,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment
65533,2007,Tents,Star,Gazer,3,E-mail,France,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camping Equipment


In [438]:
df = pd.concat(
    [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11, str12, str13, str14, str15, str16, str17,      str18, str19, str20, str21, str22, str23, str24],
    sort = True)[['Year', 'Product line', 'Product type', 'Product', 'Order method type', 'Retailer country',                       'Revenue', 'Planned revenue','Product cost', 'Quantity', 'Unit cost', 'Unit price', 'Gross profit', 'Unit sale price']]
df.head()

,Year,Product line,Product type,Product,Order method type,Retailer country,Revenue,Planned revenue,Product cost,Quantity,Unit cost,Unit price,Gross profit,Unit sale price
17514,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,5819.7,6586.16,1733.2,619,2.8,10.64,4086.5,5.105
17529,2004,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38682,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,10904.3,11363.5,2990.4,1068,2.8,10.64,7913.88,10.210
38697,2005,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United Kingdom,27987.8,28855.7,7593.6,2712,2.8,10.64,20394.2,10.320
59850,2006,Golf Equipment,Golf Accessories,Course Pro Golf and Tee Set,Sales visit,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [440]:
df.to_csv("sales.csv", index=False)